In [546]:
import requests
from datetime import datetime
import time
import pandas as pd
import geopandas as gpd
import random
from shapely.geometry import shape, Point
import numpy as np
import csv
import io
import zipfile
import xlrd
import matplotlib
import re
from itertools import combinations
from geopy.distance import geodesic

In [547]:
API_KEY = ""
ONEMAP_ACCOUNT_KEY = ""

# **Data Preprocessing**

## Subzone and Planning Area Data

### Generate list of all subzones and regions in SG

In [548]:
dataset_id = "d_8594ae9ff96d0c708bc2af633048edfb"
url = "https://api-open.data.gov.sg/v1/public/api/datasets/" + dataset_id + "/poll-download"

response = requests.get(url)
json_data = response.json()
if json_data['code'] != 0:
    print(json_data['errMsg'])
    exit(1)

url = json_data['data']['url']
response = requests.get(url)

In [549]:
subzone = []
for feature in response.json().get("features"):
    description = pd.Series(feature["properties"]["Description"])
    subzone_no = description.str.extract(r"SUBZONE_NO<\/th>\s*<td>(.*?)<\/td>").iloc[0, 0]
    subzone_name = description.str.extract(r"SUBZONE_N<\/th>\s*<td>(.*?)<\/td>").iloc[0, 0]
    subzone_code = description.str.extract(r"SUBZONE_C<\/th>\s*<td>(.*?)<\/td>").iloc[0, 0]
    planning_area_name = description.str.extract(r"PLN_AREA_N<\/th>\s*<td>(.*?)<\/td>").iloc[0, 0]
    planning_area_code = description.str.extract(r"PLN_AREA_C<\/th>\s*<td>(.*?)<\/td>").iloc[0, 0]
    region_name = description.str.extract(r"REGION_N<\/th>\s*<td>(.*?)<\/td>").iloc[0, 0]
    region_code = description.str.extract(r"REGION_C<\/th>\s*<td>(.*?)<\/td>").iloc[0, 0]
    geometry = shape(feature['geometry'])
    subzone.append({
        "subzone_no":subzone_no,
        "subzone_name":subzone_name,
        "subzone_code":subzone_code,
        "planning_area_name":planning_area_name,
        "planning_area_code":planning_area_code,
        "region_name":region_name,
        "region_code":region_code,
        "geometry":geometry
    })
subzone = gpd.GeoDataFrame(subzone, geometry="geometry", crs="EPSG:4326")

# Rename column to geog_boundary
subzone["geog_boundary"] = subzone.geometry

subzone = subzone.to_crs(epsg=3857)
subzone['area'] = subzone.geometry.area / 10**6
subzone = subzone.to_crs(epsg=4326)
subzone.head()

,subzone_no,subzone_name,subzone_code,planning_area_name,planning_area_code,region_name,region_code,geometry,geog_boundary,area
0,12,DEPOT ROAD,BMSZ12,BUKIT MERAH,BM,CENTRAL REGION,CR,"POLYGON Z ((103.81454 1.28239 0, 103.81774 1.2...","POLYGON Z ((103.81454 1.28239 0, 103.81774 1.2...",0.445498
1,2,BUKIT MERAH,BMSZ02,BUKIT MERAH,BM,CENTRAL REGION,CR,"POLYGON Z ((103.82209 1.28049 0, 103.8221 1.28...","POLYGON Z ((103.82209 1.28049 0, 103.8221 1.28...",0.414702
2,3,CHINATOWN,OTSZ03,OUTRAM,OT,CENTRAL REGION,CR,"POLYGON Z ((103.84375 1.28508 0, 103.844 1.284...","POLYGON Z ((103.84375 1.28508 0, 103.844 1.284...",0.591471
3,4,PHILLIP,DTSZ04,DOWNTOWN CORE,DT,CENTRAL REGION,CR,"POLYGON Z ((103.84962 1.28412 0, 103.84955 1.2...","POLYGON Z ((103.84962 1.28412 0, 103.84955 1.2...",0.039723
4,5,RAFFLES PLACE,DTSZ05,DOWNTOWN CORE,DT,CENTRAL REGION,CR,"POLYGON Z ((103.85253 1.28617 0, 103.85253 1.2...","POLYGON Z ((103.85253 1.28617 0, 103.85253 1.2...",0.190134


### Creating Area Dataframe to Store Area of Subzone

In [550]:
# Run to download subzone_area.csv
df_area = subzone.loc[:,['subzone_name','area']]
#df_area.to_csv('../data/subzone_area.csv',index=False)

## Distance Between Subzones

In [551]:
# Load GeoJSON into a GeoDataFrame
gdf = gpd.read_file("../data/Master Plan 2019 Subzone Boundary (No Sea) (GEOJSON).geojson")

# Display the first few rows
gdf['Description'][0]
    

# Plot the data (requires Matplotlib)
#gdf.plot()

'<center><table><tr><th colspan=\'2\' align=\'center\'><em>Attributes</em></th></tr><tr bgcolor="#E3E3F3"> <th>SUBZONE_NO</th> <td>12</td> </tr><tr bgcolor=""> <th>SUBZONE_N</th> <td>DEPOT ROAD</td> </tr><tr bgcolor="#E3E3F3"> <th>SUBZONE_C</th> <td>BMSZ12</td> </tr><tr bgcolor=""> <th>CA_IND</th> <td>N</td> </tr><tr bgcolor="#E3E3F3"> <th>PLN_AREA_N</th> <td>BUKIT MERAH</td> </tr><tr bgcolor=""> <th>PLN_AREA_C</th> <td>BM</td> </tr><tr bgcolor="#E3E3F3"> <th>REGION_N</th> <td>CENTRAL REGION</td> </tr><tr bgcolor=""> <th>REGION_C</th> <td>CR</td> </tr><tr bgcolor="#E3E3F3"> <th>INC_CRC</th> <td>C22DED671DE2A940</td> </tr><tr bgcolor=""> <th>FMEL_UPD_D</th> <td>20191223152313</td> </tr></table></center>'

In [552]:
def matchsubzone(des):
    matched = re.search(r"<th>SUBZONE_N</th> <td>(.*?)</td>", des)

    return matched.group(1)

def planningarea(des):
    matched =re.search(r"<th>PLN_AREA_N</th> <td>(.*?)</td>",des)
    return matched.group(1)

gdf['subzone'] = gdf['Description'].apply(matchsubzone)
gdf['planning_area']=gdf['Description'].apply(planningarea)

gdf.head()

,Name,Description,geometry,subzone,planning_area
0,kml_1,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((103.81454 1.28239 0, 103.81774 1.2...",DEPOT ROAD,BUKIT MERAH
1,kml_2,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((103.82209 1.28049 0, 103.8221 1.28...",BUKIT MERAH,BUKIT MERAH
2,kml_3,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((103.84375 1.28508 0, 103.844 1.284...",CHINATOWN,OUTRAM
3,kml_4,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((103.84962 1.28412 0, 103.84955 1.2...",PHILLIP,DOWNTOWN CORE
4,kml_5,<center><table><tr><th colspan='2' align='cent...,"POLYGON Z ((103.85253 1.28617 0, 103.85253 1.2...",RAFFLES PLACE,DOWNTOWN CORE


In [553]:
# Get the centroids of the subzones
gdf['centroids']= gdf.geometry.centroid
gdf.crs

gdf_new=gdf[['subzone','centroids']]
gdf_new.head()

/var/folders/2m/8llkplds47v3b1fv6f6vqv0m0000gn/T/ipykernel_11438/1106347456.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroids']= gdf.geometry.centroid


,subzone,centroids
0,DEPOT ROAD,POINT (103.80856 1.28222)
1,BUKIT MERAH,POINT (103.81859 1.28201)
2,CHINATOWN,POINT (103.84369 1.27997)
3,PHILLIP,POINT (103.84865 1.28528)
4,RAFFLES PLACE,POINT (103.85101 1.28372)


In [554]:
# Generate all possible region pairs
region_pairs = list(combinations(gdf_new.itertuples(index=False), 2))


# Compute centroid distances
data = []
for r1, r2 in region_pairs:
    c1 = (r1.centroids.y,r1.centroids.x,)
    c2 = (r2.centroids.y,r2.centroids.x,)
    distance = geodesic(c1, c2).kilometers  # Compute distance
    data.append([r1.subzone, r2.subzone, distance])
    
# Create a DataFrame with distances
df_distances = pd.DataFrame(data, columns=['Region1', 'Region2', 'Distance'])
df_distances.head()

,Region1,Region2,Distance
0,DEPOT ROAD,BUKIT MERAH,1.116521
1,DEPOT ROAD,CHINATOWN,3.917380
2,DEPOT ROAD,PHILLIP,4.474975
3,DEPOT ROAD,RAFFLES PLACE,4.727005
4,DEPOT ROAD,CHINA SQUARE,4.271837


In [555]:
# Save df_distances to a CSV file
#df_distances.to_csv("../data/subzone_distances.csv")

## Bus Data

### Bus Stop Primary Information
- coordinates
- name
- bus stop code
- description of nearest landmark

#### Retrieve Data from LTA Datamall

In [556]:
BASE_URL = "http://datamall2.mytransport.sg/ltaodataservice/BusStops"

def fetch_all_bus_stops():
    all_bus_stops = []
    skip = 0

    while True:
        url = f"{BASE_URL}?$skip={skip}" if skip > 0 else BASE_URL
        headers = {
            "AccountKey": API_KEY,
            "Accept": "application/json"
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json().get("value", [])
            if not data:
                break  # Stop if no more results
            all_bus_stops.extend(data)
            skip += 500
            time.sleep(1)  # Prevent exceeding rate limits
        else:
            print(f"Error {response.status_code}: {response.text}")
            break  # Stop if an error occurs

    return all_bus_stops

# Fetch and Create dataframe of bus stops data retrieved
bus_stops = fetch_all_bus_stops()
bus_stops_df = pd.DataFrame(bus_stops)
bus_stops_df.head()


,BusStopCode,RoadName,Description,Latitude,Longitude
0,01012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
1,01013,Victoria St,St. Joseph's Ch,1.297710,103.853225
2,01019,Victoria St,Bras Basah Cplx,1.296990,103.853022
3,01029,Nth Bridge Rd,Opp Natl Lib,1.296673,103.854414
4,01039,Nth Bridge Rd,Bugis Cube,1.298208,103.855491


#### Check and Convert Data Type

In [557]:
bus_stops_df.dtypes

BusStopCode     object
RoadName        object
Description     object
Latitude       float64
Longitude      float64
dtype: object

In [558]:
# Convert columns to integer type
bus_stops_df["BusStopCode"] = bus_stops_df["BusStopCode"].astype(int)

#### Create new Point object column to store location coordinates of bus stops

In [559]:
bus_stops_df['geometry'] = bus_stops_df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
bus_stops_df = gpd.GeoDataFrame(bus_stops_df, geometry='geometry', crs='EPSG:4326')
bus_stops_df.head()

,BusStopCode,RoadName,Description,Latitude,Longitude,geometry
0,1012,Victoria St,Hotel Grand Pacific,1.296848,103.852536,POINT (103.85254 1.29685)
1,1013,Victoria St,St. Joseph's Ch,1.297710,103.853225,POINT (103.85322 1.29771)
2,1019,Victoria St,Bras Basah Cplx,1.296990,103.853022,POINT (103.85302 1.29699)
3,1029,Nth Bridge Rd,Opp Natl Lib,1.296673,103.854414,POINT (103.85441 1.29667)
4,1039,Nth Bridge Rd,Bugis Cube,1.298208,103.855491,POINT (103.85549 1.29821)


In [560]:
#Run this to save the bus_stops data
#bus_stops_df.to_csv("../data/bus_stops_df.csv")

### Bus Services Data

#### Retrieve Data from LTA Datamall

In [561]:
BASE_URL = "http://datamall2.mytransport.sg/ltaodataservice/BusServices"

def fetch_all_bus_services():
    all_bus_services = []
    skip = 0

    while True:
        url = f"{BASE_URL}?$skip={skip}" if skip > 0 else BASE_URL
        headers = {
            "AccountKey": API_KEY,
            "Accept": "application/json"
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json().get("value", [])
            if not data:
                break  # Stop if no more results
            all_bus_services.extend(data)
            skip += 500
            time.sleep(1)  # Prevent exceeding rate limits
        else:
            print(f"Error {response.status_code}: {response.text}")
            break  # Stop if an error occurs

    return all_bus_services

# Fetch and Create dataframe of bus services data retrieved
bus_services = fetch_all_bus_services()
bus_services_df = pd.DataFrame(bus_services)
bus_services_df.head()

,ServiceNo,Operator,Direction,Category,OriginCode,DestinationCode,AM_Peak_Freq,AM_Offpeak_Freq,PM_Peak_Freq,PM_Offpeak_Freq,LoopDesc
0,118,GAS,1,TRUNK,65009,97009,5-08,8-12,8-10,09-14,
1,118,GAS,2,TRUNK,97009,65009,10-10,8-11,4-08,9-12,
2,118A,GAS,1,TRUNK,65009,96119,06-66,-,-,-,
3,118B,GAS,1,TRUNK,96111,65191,-,-,24-57,-,
4,119,GAS,1,TRUNK,65009,65009,09-13,12-18,12-15,15-17,Hougang St 21


#### Check and Convert Data Type

In [562]:
bus_services_cleaned = bus_services_df.replace("-", np.nan) # For cells with no , convert to NA

freq_cols = ["AM_Peak_Freq", "AM_Offpeak_Freq", "PM_Peak_Freq", "PM_Offpeak_Freq"]
for col in freq_cols:
    bus_services_cleaned[[f"{col.replace('Freq', '')}Min_Interval", f"{col.replace('Freq', '')}Max_Interval"]] = (bus_services_cleaned[col]
                                                                                                                  .str.split("-", expand=True)
                                                                                                                  .apply(pd.to_numeric,errors = "coerce")
                                                                                                )
bus_services_cleaned = bus_services_cleaned.drop(columns=freq_cols)
bus_services_cleaned = bus_services_cleaned.iloc[:, [0,7,8,9,10,11,12,13,14]]

for col in bus_services_cleaned.columns[1:]:
        upper_pct = np.nanpercentile(bus_services_cleaned[col].dropna(), 99)
        bus_services_cleaned[col].fillna(upper_pct, inplace = True)

bus_services_cleaned.groupby(["ServiceNo"]).mean().reset_index()
bus_services_cleaned.head()

/var/folders/2m/8llkplds47v3b1fv6f6vqv0m0000gn/T/ipykernel_11438/2039829874.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  bus_services_cleaned[col].fillna(upper_pct, inplace = True)


,ServiceNo,AM_Peak_Min_Interval,AM_Peak_Max_Interval,AM_Offpeak_Min_Interval,AM_Offpeak_Max_Interval,PM_Peak_Min_Interval,PM_Peak_Max_Interval,PM_Offpeak_Min_Interval,PM_Offpeak_Max_Interval
0,118,5.00,8.00,8.0,12.0,8.0,10.00,9.0,14.00
1,118,10.00,10.00,8.0,11.0,4.0,8.00,9.0,12.00
2,118A,6.00,66.00,30.0,47.5,31.6,46.47,30.0,41.57
3,118B,38.46,52.96,30.0,47.5,24.0,57.00,30.0,41.57
4,119,9.00,13.00,12.0,18.0,12.0,15.00,15.0,17.00


In [563]:
bus_services_cleaned.dtypes

ServiceNo                   object
AM_Peak_Min_Interval       float64
AM_Peak_Max_Interval       float64
AM_Offpeak_Min_Interval    float64
AM_Offpeak_Max_Interval    float64
PM_Peak_Min_Interval       float64
PM_Peak_Max_Interval       float64
PM_Offpeak_Min_Interval    float64
PM_Offpeak_Max_Interval    float64
dtype: object

### Bus Routes Data

#### Retrieve Data from LTA Datamall

In [564]:
BASE_URL = "http://datamall2.mytransport.sg/ltaodataservice/BusRoutes"

def fetch_all_bus_routes():
    all_bus_routes = []
    skip = 0

    while True:
        url = f"{BASE_URL}?$skip={skip}" if skip > 0 else BASE_URL
        headers = {
            "AccountKey": API_KEY,
            "Accept": "application/json"
        }

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json().get("value", [])
            if not data:
                break  # Stop if no more results
            all_bus_routes.extend(data)
            skip += 500
            time.sleep(1)  # Prevent exceeding rate limits
        else:
            print(f"Error {response.status_code}: {response.text}")
            break  # Stop if an error occurs

    return all_bus_routes

# Fetch and Create dataframe of bus routes data retrieved
bus_routes = fetch_all_bus_routes()
bus_routes_df = pd.DataFrame(bus_routes)
bus_routes_df.head()
#Run this to save the routes data
#bus_routes_df.to_csv("../data/bus_routes_df.csv")


In [565]:
bus_routes_df = pd.read_csv("../data/bus_routes_df.csv", index_col=0)
bus_routes_df.head()

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus
0,10,SBST,1,1,75009,0.0,0500,2300,0500,2300,0500,2300
1,10,SBST,1,2,76059,0.6,0502,2302,0502,2302,0502,2302
2,10,SBST,1,3,76069,1.1,0504,2304,0504,2304,0503,2304
3,10,SBST,1,4,96289,2.3,0508,2308,0508,2309,0507,2308
4,10,SBST,1,5,96109,2.7,0509,2310,0509,2311,0508,2309


#### Check and Convert Data Type

In [566]:
bus_routes_df.dtypes

ServiceNo        object
Operator         object
Direction         int64
StopSequence      int64
BusStopCode       int64
Distance        float64
WD_FirstBus      object
WD_LastBus       object
SAT_FirstBus     object
SAT_LastBus      object
SUN_FirstBus     object
SUN_LastBus      object
dtype: object

In [567]:
# Convert columns to integer type
bus_routes_df["BusStopCode"] = bus_routes_df["BusStopCode"].astype(int)

### Passenger Volume By Origin Destination Bus Stops

In [568]:
bus_pvod_202411 = pd.read_csv("../data/origin_destination_bus_202411.csv")
bus_pvod_202412 = pd.read_csv("../data/origin_destination_bus_202412.csv")
bus_pvod_202501 = pd.read_csv("../data/origin_destination_bus_202501.csv")

bus_pvod_volume = pd.concat([bus_pvod_202411, bus_pvod_202412, bus_pvod_202501], axis=0, ignore_index=True)
bus_pvod_volume.head()

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
0,2024-11,WEEKENDS/HOLIDAY,8,BUS,66331,70259,11
1,2024-11,WEEKDAY,14,BUS,20289,17319,7
2,2024-11,WEEKDAY,14,BUS,82049,60219,3
3,2024-11,WEEKDAY,11,BUS,55259,60081,3
4,2024-11,WEEKDAY,18,BUS,84659,94079,1


In [569]:
bus_pvod_volume.dtypes

YEAR_MONTH             object
DAY_TYPE               object
TIME_PER_HOUR           int64
PT_TYPE                object
ORIGIN_PT_CODE          int64
DESTINATION_PT_CODE     int64
TOTAL_TRIPS             int64
dtype: object

### Passenger Volume By Bus Stops

In [570]:
bus_pv_202411 = pd.read_csv("../data/transport_node_bus_202411.csv")
bus_pv_202412 = pd.read_csv("../data/transport_node_bus_202412.csv")
bus_pv_202501 = pd.read_csv("../data/transport_node_bus_202501.csv")

bus_pv_volume = pd.concat([bus_pv_202501, bus_pv_202412, bus_pv_202411], axis=0, ignore_index=True)
bus_pv_volume.head()

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME
0,2025-01,WEEKDAY,19.0,BUS,75419,185,530
1,2025-01,WEEKDAY,17.0,BUS,46711,1066,133
2,2025-01,WEEKENDS/HOLIDAY,20.0,BUS,43459,93,81
3,2025-01,WEEKENDS/HOLIDAY,22.0,BUS,17069,350,147
4,2025-01,WEEKENDS/HOLIDAY,14.0,BUS,47579,343,203


In [571]:
bus_pv_volume.dtypes

YEAR_MONTH               object
DAY_TYPE                 object
TIME_PER_HOUR           float64
PT_TYPE                  object
PT_CODE                   int64
TOTAL_TAP_IN_VOLUME       int64
TOTAL_TAP_OUT_VOLUME      int64
dtype: object

### Join Subzone to Bus Stop

In [572]:
bus_stops_new = gpd.sjoin(bus_stops_df, subzone, how = 'left', predicate = 'within')
bus_stops_new.drop(columns=['index_right'], inplace=True)
bus_stops_new.head()

,BusStopCode,RoadName,Description,Latitude,Longitude,geometry,subzone_no,subzone_name,subzone_code,planning_area_name,planning_area_code,region_name,region_code,geog_boundary,area
0,1012,Victoria St,Hotel Grand Pacific,1.296848,103.852536,POINT (103.85254 1.29685),10,VICTORIA,RCSZ10,ROCHOR,RC,CENTRAL REGION,CR,"POLYGON Z ((103.85404 1.30269 0, 103.8541 1.30...",0.215804
1,1013,Victoria St,St. Joseph's Ch,1.297710,103.853225,POINT (103.85322 1.29771),10,VICTORIA,RCSZ10,ROCHOR,RC,CENTRAL REGION,CR,"POLYGON Z ((103.85404 1.30269 0, 103.8541 1.30...",0.215804
2,1019,Victoria St,Bras Basah Cplx,1.296990,103.853022,POINT (103.85302 1.29699),1,BUGIS,DTSZ01,DOWNTOWN CORE,DT,CENTRAL REGION,CR,"POLYGON Z ((103.85645 1.29668 0, 103.8564 1.29...",0.282206
3,1029,Nth Bridge Rd,Opp Natl Lib,1.296673,103.854414,POINT (103.85441 1.29667),1,BUGIS,DTSZ01,DOWNTOWN CORE,DT,CENTRAL REGION,CR,"POLYGON Z ((103.85645 1.29668 0, 103.8564 1.29...",0.282206
4,1039,Nth Bridge Rd,Bugis Cube,1.298208,103.855491,POINT (103.85549 1.29821),1,BUGIS,DTSZ01,DOWNTOWN CORE,DT,CENTRAL REGION,CR,"POLYGON Z ((103.85645 1.29668 0, 103.8564 1.29...",0.282206


#### Clean Merged Data
Check rows with NaN
- Bus stops in these rows are not located in SG. Therefore we remove them.

In [573]:
bus_stops_new_with_na = bus_stops_new[bus_stops_new.isna().any(axis=1)]
bus_stops_new_with_na

,BusStopCode,RoadName,Description,Latitude,Longitude,geometry,subzone_no,subzone_name,subzone_code,planning_area_name,planning_area_code,region_name,region_code,geog_boundary,area
2324,46211,Johor Bahru,Johor Bahru Checkpt,1.464917,103.765477,POINT (103.76548 1.46492),NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
2325,46219,Johor Bahru,Johor Bahru Checkpt,1.465427,103.768267,POINT (103.76827 1.46543),NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
2328,46239,Jln Datin Halimah,Larkin Ter,1.493904,103.744033,POINT (103.74403 1.4939),NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN
2529,47711,Jln Jim Quee,JB Sentral Bus Ter,1.463458,103.764423,POINT (103.76442 1.46346),NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN


In [574]:
# Removing all rows containing NA values
bus_stops_new = bus_stops_new.dropna()

Check for duplicated data
- No duplicated data found

In [575]:
# Check for duplicate rows in the dataset
model_duplicates = bus_stops_new.duplicated().sum()
model_duplicates

0

## Aggregate Data on a Bus Stop Level
- Number of unique bus services per stop
- Mean distance covered by buses per bus stop (from total distance covered per bus service)

In [576]:
bus_services_per_stop = pd.DataFrame(
        bus_routes_df.groupby("BusStopCode")["ServiceNo"] #finds total unique bus services per bus stop
                    .nunique()
                    .reset_index()
                    .rename(columns = {"ServiceNo": "num_bus_services"})) 

dist_per_svc = pd.DataFrame( #find the total distance travelled per bus service
    bus_routes_df.groupby("ServiceNo")
                .agg({"Distance":"max"})
                .reset_index()
                .rename(columns = {"Distance": "Total_Distance"}))

bus_routes_df = bus_routes_df.merge(dist_per_svc, on = "ServiceNo")

info_per_svc = bus_routes_df.merge(bus_services_cleaned, on="ServiceNo")

mean_dist_per_stop = pd.DataFrame( # mean intervals at different timings of the day
        info_per_svc.groupby("BusStopCode")
                    .agg({"Total_Distance": "mean",
                        "AM_Peak_Max_Interval": "mean",
                        "AM_Peak_Min_Interval": "mean",
                        "AM_Offpeak_Min_Interval": "mean",
                        "AM_Offpeak_Max_Interval": "mean",
                        "PM_Peak_Min_Interval": "mean",
                        "PM_Peak_Max_Interval": "mean",
                        "PM_Offpeak_Min_Interval": "mean",
                        "PM_Offpeak_Max_Interval": "mean"})
                    .rename(columns = {"Total_Distance": "mean_bus_distance"})
                    .reset_index()
)

bus_services_per_stop = bus_services_per_stop.merge(mean_dist_per_stop, on = "BusStopCode")
bus_stops_acc = bus_stops_new.merge(bus_services_per_stop, on = "BusStopCode", how = "left")

## Aggregate bus data on a subzone level

In [577]:
bus_stops_agg = pd.DataFrame(
        bus_stops_acc.groupby(["subzone_code", "planning_area_name", "subzone_name", "geog_boundary"])
                    .agg({"BusStopCode":"count",
                          "num_bus_services":"sum",
                          "mean_bus_distance": "mean",
                        "AM_Peak_Max_Interval": "mean",
                        "AM_Peak_Min_Interval": "mean",
                        "AM_Offpeak_Min_Interval": "mean",
                        "AM_Offpeak_Max_Interval": "mean",
                        "PM_Peak_Min_Interval": "mean",
                        "PM_Peak_Max_Interval": "mean",
                        "PM_Offpeak_Min_Interval": "mean",
                        "PM_Offpeak_Max_Interval": "mean"})
                    .rename(columns = {"BusStopCode": "num_bus_stops"})
                    .reset_index()
)
bus_stops_agg.head()

,subzone_code,planning_area_name,subzone_name,geog_boundary,num_bus_stops,num_bus_services,mean_bus_distance,AM_Peak_Max_Interval,AM_Peak_Min_Interval,AM_Offpeak_Min_Interval,AM_Offpeak_Max_Interval,PM_Peak_Min_Interval,PM_Peak_Max_Interval,PM_Offpeak_Min_Interval,PM_Offpeak_Max_Interval
0,AMSZ01,ANG MO KIO,ANG MO KIO TOWN CENTRE,"POLYGON Z ((103.84853 1.36879 0, 103.84834 1.3...",9,89,19.715117,16.883207,11.822818,12.778459,18.626422,12.963205,17.289356,14.928982,20.624938
1,AMSZ02,ANG MO KIO,CHENG SAN,"POLYGON Z ((103.85474 1.37583 0, 103.85524 1.3...",21,110,18.212157,17.336965,11.997247,14.474189,24.414430,13.698242,19.270679,16.460606,23.618533
2,AMSZ03,ANG MO KIO,CHONG BOON,"POLYGON Z ((103.85737 1.36904 0, 103.85813 1.3...",15,108,18.580268,14.622437,10.737684,12.339644,19.342175,11.915586,15.698547,14.150056,19.209948
3,AMSZ04,ANG MO KIO,TOWNSVILLE,"POLYGON Z ((103.84814 1.36879 0, 103.84834 1.3...",9,72,19.004196,14.921737,11.554133,10.984058,16.715742,11.179162,15.386525,13.188133,18.236091
4,AMSZ05,ANG MO KIO,SHANGRI-LA,"POLYGON Z ((103.83537 1.37093 0, 103.83576 1.3...",12,71,17.391642,16.639670,12.536028,12.035482,20.316981,11.724192,18.169165,12.965820,19.291760


## MRT Data

### Scrape MRT Stations Data
- geographical coordinates of each mrt/lrt station
- name of each mrt/lrt station


Scraping method based on https://github.com/elliotwutingfeng/singapore_train_station_coordinates/blob/main/station_coordinates.pyv

In [578]:
# Run this to download mrt.csv
STATION_DATA_ENDPOINT = (
    "https://datamall.lta.gov.sg/content/dam/datamall/datasets/Geospatial/"
    "Train%20Station%20Codes%20and%20Chinese%20Names.zip"
)

def get_station_names(endpoint: str) -> list[tuple[str, str]]:
    """Download train station codes and station names.

    Args:
        endpoint (str): HTTPS address of zipped XLS file containing train station codes and names.

    Returns:
        list[tuple[str, str]]: Train stations sorted by station code in ascending order.
        For example, ("CC1", "Dhoby Ghaut"), ("NE6", "Dhoby Ghaut"), ("NS24", "Dhoby Ghaut").
    """
    with requests.Session() as session:
        res = session.get(endpoint, timeout=30)
        res.raise_for_status()
    with zipfile.ZipFile(io.BytesIO(res.content), "r") as z:
        excel_bytes = z.read(
            z.infolist()[0]
        )  # Zip file should only contain one XLS file.
        workbook = xlrd.open_workbook(file_contents=excel_bytes)
        sheet = workbook.sheet_by_index(0)

    stations: set[tuple[str, str]] = {
        (sheet.cell_value(row_idx, 0).strip(), sheet.cell_value(row_idx, 1).strip())
        for row_idx in range(1, sheet.nrows)
    }

    return sorted(
        stations,
        key=lambda station: station[0],
    )


def get_coordinates_onemap(location_name):
    endpoint = "https://www.onemap.gov.sg/api/common/elastic/search"

    res = requests.get(
        endpoint,
        params={
            "searchVal": location_name,
            "returnGeom": "Y",
            "getAddrDetails": "Y",
            "pageNum": "1",
        },
        timeout=15,
    ).json()
    results = res.get("results", None)
    if isinstance(results, list):
        for result in results:
            if "LATITUDE" in result and "LONGITUDE" in result:
                return float(result["LATITUDE"]), float(result["LONGITUDE"])
    return None

stations = {
        station: {
            "lat": None,
            "lon": None,
        }
        for station in get_station_names(STATION_DATA_ENDPOINT)
    }

# Get all operational stations coordinates from OneMap.
for (station_code, station_name), station_details in stations.items():
    coordinates = None
    location_name = station_name + " " + station_code
    try:
        coordinates = get_coordinates_onemap(location_name)
        if coordinates:
            stations[(station_code, station_name)]["lat"] = coordinates[0]
            stations[(station_code, station_name)]["lon"] = coordinates[1]
    except Exception as e:
        _ = e

In [579]:
## Run this to download mrt.csv
with open("../data/mrt.csv", "w") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(
        ("station_code", "station_name", "lat", "lon")
    )
    csv_writer.writerows(
        sorted(
            (
                (
                    station_code,
                    station_name,
                    *details.values(),
                )
                for (
                    station_code,
                    station_name,
                ), details in stations.items()
            ),
            key=lambda x: x[0])
        )

In [580]:
mrt = pd.read_csv("../data/mrt.csv", index_col = 0).reset_index()
mrt.head()

,station_code,station_name,lat,lon
0,BP1,Choa Chu Kang,1.384755,103.744538
1,BP10,Fajar,1.384573,103.770887
2,BP11,Segar,1.387785,103.769600
3,BP12,Jelapang,1.386739,103.764534
4,BP13,Senja,1.382725,103.762344


### Passenger Volume By Origin Destination Train

In [581]:
train_pvod_202411 = pd.read_csv("../data/origin_destination_train_202411.csv")
train_pvod_202412 = pd.read_csv("../data/origin_destination_train_202412.csv")
train_pvod_202501 = pd.read_csv("../data/origin_destination_train_202501.csv")

train_pvod_volume = pd.concat([train_pvod_202411, train_pvod_202412, train_pvod_202501], axis=0, ignore_index=True)
train_pvod_volume.head()

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
0,2024-11,WEEKENDS/HOLIDAY,19,TRAIN,TE22,CC11,3
1,2024-11,WEEKENDS/HOLIDAY,15,TRAIN,NE14,TE5,1
2,2024-11,WEEKDAY,12,TRAIN,NS4/BP1,BP5,232
3,2024-11,WEEKDAY,22,TRAIN,CC20,CC26,5
4,2024-11,WEEKDAY,20,TRAIN,DT29,EW5,1


In [582]:
train_pvod_volume['ORIGIN_PT_CODE'] = train_pvod_volume['ORIGIN_PT_CODE'].str.split('/').str[0]
train_pvod_volume['DESTINATION_PT_CODE'] = train_pvod_volume['DESTINATION_PT_CODE'].str.split('/').str[0]
train_pvod_volume.head()

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS
0,2024-11,WEEKENDS/HOLIDAY,19,TRAIN,TE22,CC11,3
1,2024-11,WEEKENDS/HOLIDAY,15,TRAIN,NE14,TE5,1
2,2024-11,WEEKDAY,12,TRAIN,NS4,BP5,232
3,2024-11,WEEKDAY,22,TRAIN,CC20,CC26,5
4,2024-11,WEEKDAY,20,TRAIN,DT29,EW5,1


In [583]:
train_pvod_volume.dtypes

YEAR_MONTH             object
DAY_TYPE               object
TIME_PER_HOUR           int64
PT_TYPE                object
ORIGIN_PT_CODE         object
DESTINATION_PT_CODE    object
TOTAL_TRIPS             int64
dtype: object

### Passenger Volume By Train

In [584]:
train_pv_202411 = pd.read_csv("../data/transport_node_train_202411.csv")
train_pv_202412 = pd.read_csv("../data/transport_node_train_202412.csv")
train_pv_202501 = pd.read_csv("../data/transport_node_train_202501.csv")

train_pv_volume = pd.concat([train_pv_202411, train_pv_202412, train_pv_202501], axis=0, ignore_index=True)
train_pv_volume.head()

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME
0,2024-11,WEEKDAY,8,TRAIN,CC8,25245,10368
1,2024-11,WEEKENDS/HOLIDAY,15,TRAIN,CC23,2160,1721
2,2024-11,WEEKENDS/HOLIDAY,17,TRAIN,DT10,2169,1570
3,2024-11,WEEKDAY,11,TRAIN,PE1,3935,2012
4,2024-11,WEEKDAY,10,TRAIN,TE3,9716,4942


In [585]:
train_pv_volume['PT_CODE'] = train_pv_volume['PT_CODE'].str.split('/').str[0]
train_pv_volume.head()

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME
0,2024-11,WEEKDAY,8,TRAIN,CC8,25245,10368
1,2024-11,WEEKENDS/HOLIDAY,15,TRAIN,CC23,2160,1721
2,2024-11,WEEKENDS/HOLIDAY,17,TRAIN,DT10,2169,1570
3,2024-11,WEEKDAY,11,TRAIN,PE1,3935,2012
4,2024-11,WEEKDAY,10,TRAIN,TE3,9716,4942


In [586]:
train_pv_volume.dtypes

YEAR_MONTH              object
DAY_TYPE                object
TIME_PER_HOUR            int64
PT_TYPE                 object
PT_CODE                 object
TOTAL_TAP_IN_VOLUME      int64
TOTAL_TAP_OUT_VOLUME     int64
dtype: object

### Join Subzone to MRT

In [587]:
mrt['geometry']= mrt.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
mrt = gpd.GeoDataFrame(mrt, geometry='geometry', crs='EPSG:4326')
# Convert to a projected CRS (Singapore SVY21 - EPSG:3414)
mrt = mrt.to_crs(epsg=3414)
subzone = subzone.to_crs(epsg=3414)  # Ensure subzone is in the same CRS

mrt = gpd.sjoin_nearest(mrt, subzone, how = 'left')

# Convert back to WGS84 if needed
mrt = mrt.to_crs(epsg=4326)

mrt.head()

,station_code,station_name,lat,lon,geometry,index_right,subzone_no,subzone_name,subzone_code,planning_area_name,planning_area_code,region_name,region_code,geog_boundary,area
0,BP1,Choa Chu Kang,1.384755,103.744538,POINT (103.74454 1.38476),244,3,CHOA CHU KANG CENTRAL,CKSZ03,CHOA CHU KANG,CK,WEST REGION,WR,"POLYGON Z ((103.75265 1.38286 0, 103.7526 1.38...",1.103706
1,BP10,Fajar,1.384573,103.770887,POINT (103.77089 1.38457),245,3,FAJAR,BPSZ03,BUKIT PANJANG,BP,WEST REGION,WR,"POLYGON Z ((103.77499 1.3845 0, 103.77465 1.38...",0.673347
2,BP11,Segar,1.387785,103.769600,POINT (103.7696 1.38779),247,2,SAUJANA,BPSZ02,BUKIT PANJANG,BP,WEST REGION,WR,"POLYGON Z ((103.76712 1.38215 0, 103.76708 1.3...",0.679933
3,BP12,Jelapang,1.386739,103.764534,POINT (103.76453 1.38674),247,2,SAUJANA,BPSZ02,BUKIT PANJANG,BP,WEST REGION,WR,"POLYGON Z ((103.76712 1.38215 0, 103.76708 1.3...",0.679933
4,BP13,Senja,1.382725,103.762344,POINT (103.76234 1.38273),246,1,SENJA,BPSZ01,BUKIT PANJANG,BP,WEST REGION,WR,"POLYGON Z ((103.76441 1.38887 0, 103.76435 1.3...",0.629451


#### Clean Merged Data

In [590]:
# Check for NaN
mrt_with_na = mrt[mrt.isna().any(axis=1)]
mrt_with_na

,station_code,station_name,lat,lon,geometry,index_right,subzone_no,subzone_name,subzone_code,planning_area_name,planning_area_code,region_name,region_code,geog_boundary,area


In [591]:
for i in stations.keys():
    if i[1]=="Orchard":
        print(i)

('NS22', 'Orchard')
('TE14', 'Orchard')


# **Commuter Hub Score Data**

## Bus Inflow and Outflow
Creating Inflow and Outflow score where score = total trips * distance

In [592]:
# Merge to get destination_subzone
bus_pvod_volume_new = bus_pvod_volume.merge(
    bus_stops_new[["BusStopCode", "subzone_name"]],
    left_on="DESTINATION_PT_CODE",
    right_on="BusStopCode",
    how="left"
).rename(columns={"subzone_name": "destination_subzone"})

# Merge again to get origin_subzone
bus_pvod_volume_new = bus_pvod_volume_new.merge(
    bus_stops_new[["BusStopCode", "subzone_name"]],
    left_on="ORIGIN_PT_CODE",
    right_on="BusStopCode",
    how="left"
).rename(columns={"subzone_name": "origin_subzone"})

bus_pvod_volume_grouped = (
    bus_pvod_volume_new
    .groupby(['YEAR_MONTH','destination_subzone','origin_subzone'])
    .agg(total_trips=("TOTAL_TRIPS","sum"))
    .reset_index()
    .rename(columns={"YEAR_MONTH": "year_month"}) 
)

bus_pvod_volume_grouped.head()

,year_month,destination_subzone,origin_subzone,total_trips
0,2024-11,ADMIRALTY,ADMIRALTY,12728
1,2024-11,ADMIRALTY,ANG MO KIO TOWN CENTRE,369
2,2024-11,ADMIRALTY,BENCOOLEN,31
3,2024-11,ADMIRALTY,BOULEVARD,118
4,2024-11,ADMIRALTY,BRAS BASAH,28


In [593]:
df_distances = pd.read_csv("../data/subzone_distances.csv")
# Remove inter-subzone trips
inflow = bus_pvod_volume_grouped[bus_pvod_volume_grouped['destination_subzone']!=bus_pvod_volume_grouped['origin_subzone']]

# Merge considering both orders of Region1 and Region2
merged_df = pd.merge(inflow, df_distances, left_on=['destination_subzone', 'origin_subzone'], right_on=['Region1', 'Region2'], how='left')

# Also consider reversed order since the dataset does not contain bidirectional distances
reversed_merged_df = pd.merge(inflow, df_distances, left_on=['destination_subzone', 'origin_subzone'], right_on=['Region2', 'Region1'], how='left')

# Combine results
final_merged_df = pd.concat([merged_df, reversed_merged_df]).dropna()
final_merged_df= final_merged_df.drop(final_merged_df.columns[[4,5,6]], axis = 1)
inflow_score = final_merged_df.sort_values(by=['year_month','destination_subzone','total_trips','origin_subzone']).reset_index(drop=True)
inflow_score['trip_x_distance']= inflow_score['total_trips'] * inflow_score['Distance']

inflow_score.head()

,year_month,destination_subzone,origin_subzone,total_trips,Distance,trip_x_distance
0,2024-11,ADMIRALTY,GOODWOOD PARK,2,15.492004,30.984008
1,2024-11,ADMIRALTY,HENDERSON HILL,6,18.402538,110.415228
2,2024-11,ADMIRALTY,TIONG BAHRU STATION,9,18.727703,168.549323
3,2024-11,ADMIRALTY,BUKIT MERAH,11,18.963732,208.601049
4,2024-11,ADMIRALTY,RAFFLES PLACE,12,18.980402,227.764827


In [594]:
# Inflow score aggregation by destination_subzone
inflow_score_bus = inflow_score.groupby(['year_month','destination_subzone']).agg(inflow_bus_score = ("trip_x_distance", "sum")).sort_values(by="inflow_bus_score", ascending=False).reset_index()
inflow_score_bus.head()

,year_month,destination_subzone,inflow_bus_score
0,2025-01,TAMPINES EAST,9.709885e+06
1,2024-11,TAMPINES EAST,9.421312e+06
2,2024-12,TAMPINES EAST,9.271430e+06
3,2024-12,CHANGI AIRPORT,8.120046e+06
4,2024-11,WOODLANDS REGIONAL CENTRE,7.706747e+06


In [595]:
# Outflow score aggregation by destination_subzone
outflow_score_bus = inflow_score.groupby(['year_month','origin_subzone']).agg(outflow_bus_score = ("trip_x_distance", "sum")).sort_values(by="outflow_bus_score", ascending=False).reset_index()
outflow_score_bus.head()

,year_month,origin_subzone,outflow_bus_score
0,2025-01,TAMPINES EAST,9.927766e+06
1,2024-11,TAMPINES EAST,9.629060e+06
2,2024-12,TAMPINES EAST,9.515948e+06
3,2024-11,WOODLANDS REGIONAL CENTRE,8.303661e+06
4,2025-01,WOODLANDS REGIONAL CENTRE,8.163677e+06


## Total Bus Tap In and Tap Out

In [596]:
bus_pv_volume_grouped = (bus_pv_volume.groupby(["PT_CODE","YEAR_MONTH"])[['TOTAL_TAP_IN_VOLUME','TOTAL_TAP_OUT_VOLUME']]
                           .sum()
                           .reset_index()
                           .rename(columns={"PT_CODE": "BusStopCode", "TOTAL_TAP_IN_VOLUME": "total_tap_in_volume", "TOTAL_TAP_OUT_VOLUME": "total_tap_out_volume", "YEAR_MONTH": "year_month"}))
bus_pv_volume_grouped.head()

,BusStopCode,year_month,total_tap_in_volume,total_tap_out_volume
0,1012,2024-11,28528,37033
1,1012,2024-12,28630,37328
2,1012,2025-01,28081,37185
3,1013,2024-11,26484,16559
4,1013,2024-12,23912,16605


## Bus Frequency Data

In [597]:
# Merge bus routes and bus services
bus_freq_df = bus_routes_df.merge(bus_services_df, on=["ServiceNo", "Direction"], how="left")

# Select specific columns
selected_columns = ["ServiceNo", "Direction", "BusStopCode", "WD_FirstBus", "WD_LastBus",  "SAT_FirstBus",  "SAT_LastBus",  "SUN_FirstBus", "SUN_LastBus",  "AM_Peak_Freq","AM_Offpeak_Freq", "PM_Peak_Freq", "PM_Offpeak_Freq"]  # Adjust columns as needed
bus_freq_df = bus_freq_df[selected_columns]
bus_freq_df.head()

,ServiceNo,Direction,BusStopCode,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus,AM_Peak_Freq,AM_Offpeak_Freq,PM_Peak_Freq,PM_Offpeak_Freq
0,10,1,75009,0500,2300,0500,2300,0500,2300,09-09,07-16,10-17,15-17
1,10,1,76059,0502,2302,0502,2302,0502,2302,09-09,07-16,10-17,15-17
2,10,1,76069,0504,2304,0504,2304,0503,2304,09-09,07-16,10-17,15-17
3,10,1,96289,0508,2308,0508,2309,0507,2308,09-09,07-16,10-17,15-17
4,10,1,96109,0509,2310,0509,2311,0508,2309,09-09,07-16,10-17,15-17


In [598]:
# Function to process frequency ranges and compute buses per hour
def process_freq_range(freq_range):
    if isinstance(freq_range, str) and '-' in freq_range:
        # Split the range into two numbers
        freq_values = freq_range.split('-')
        try:
            # Convert to integers
            min_freq, max_freq = int(freq_values[0]), int(freq_values[1])
            avg_freq = (min_freq + max_freq) / 2  # Take the average
            return round(60 / avg_freq) if avg_freq > 0 else 0  # Convert to buses per hour
        except ValueError:
            return np.nan  # Handle conversion failure
    return np.nan  # Return NaN for invalid data

# List of frequency columns to process
freq_columns = ["AM_Peak_Freq", "AM_Offpeak_Freq", "PM_Peak_Freq", "PM_Offpeak_Freq"]

# Apply the processing function to all frequency columns
for col in freq_columns:
    bus_freq_df[col] = bus_freq_df[col].apply(process_freq_range).astype(pd.Int64Dtype())  # Convert to integer type

# Compute total buses per hour by summing across all periods
bus_freq_df["Buses_Per_Hour"] = (bus_freq_df["AM_Peak_Freq"] * 2 + bus_freq_df["AM_Offpeak_Freq"] * 8 + bus_freq_df["PM_Peak_Freq"] * 2 + bus_freq_df["PM_Offpeak_Freq"] * 4)/16

# Group by BusStopCode and sum across all services
bus_stop_hourly_buses = bus_freq_df.groupby("BusStopCode")["Buses_Per_Hour"].sum().reset_index()

# Display the first few rows
bus_stop_hourly_buses.head()


,BusStopCode,Buses_Per_Hour
0,1012,47.0
1,1013,33.375
2,1019,30.625
3,1029,49.375
4,1039,49.375


## MRT Inflow and Outflow
Creating Inflow and Outflow score where score = total trips * distance

In [599]:
# Merge to get destination_subzone
train_pvod_volume_new = train_pvod_volume.merge(
    mrt[["station_code", "subzone_name"]],
    left_on="DESTINATION_PT_CODE",
    right_on="station_code",
    how="left"
).rename(columns={"subzone_name": "destination_subzone"})

# Merge again to get origin_subzone
train_pvod_volume_new = train_pvod_volume_new.merge(
    mrt[["station_code", "subzone_name"]],
    left_on="ORIGIN_PT_CODE",
    right_on="station_code",
    how="left"
).rename(columns={"subzone_name": "origin_subzone"})

train_pvod_volume_grouped = (
    train_pvod_volume_new
    .groupby(["origin_subzone","destination_subzone", "YEAR_MONTH"])
    .agg(
        total_trips=("TOTAL_TRIPS", "sum")
    )
    .reset_index()
).rename(columns={"YEAR_MONTH": "year_month"})

train_pvod_volume_grouped.head()

,origin_subzone,destination_subzone,year_month,total_trips
0,ALJUNIED,ALJUNIED,2024-11,11310
1,ALJUNIED,ALJUNIED,2024-12,8772
2,ALJUNIED,ALJUNIED,2025-01,14542
3,ALJUNIED,ANAK BUKIT,2024-11,4228
4,ALJUNIED,ANAK BUKIT,2024-12,3719


In [600]:
df_distances = pd.read_csv("../data/subzone_distances.csv")

# Remove inter-subzone trips
inflow_train = train_pvod_volume_grouped[train_pvod_volume_grouped['destination_subzone']!=train_pvod_volume_grouped['origin_subzone']]

# Merge considering both orders of Region1 and Region2
merged_df = pd.merge(inflow_train, df_distances, left_on=['destination_subzone', 'origin_subzone'], right_on=['Region1', 'Region2'], how='left')

# Also consider reversed order since the dataset does not contain bidirectional distances
reversed_merged_df = pd.merge(inflow_train, df_distances, left_on=['destination_subzone', 'origin_subzone'], right_on=['Region2', 'Region1'], how='left')

# Combine results
final_merged_df = pd.concat([merged_df, reversed_merged_df]).dropna()
final_merged_df= final_merged_df.drop(final_merged_df.columns[[4,5,6]], axis = 1)
inflow_train_score = final_merged_df.sort_values(by=['year_month','destination_subzone','total_trips','origin_subzone']).reset_index(drop=True)
inflow_train_score['trip_x_distance']= inflow_train_score['total_trips'] * inflow_train_score['Distance']

inflow_train_score.head()

,origin_subzone,destination_subzone,year_month,total_trips,Distance,trip_x_distance
0,FAJAR,ALJUNIED,2024-11,12,14.732457,176.789478
1,BANGKIT,ALJUNIED,2024-11,19,13.997238,265.947515
2,SAUJANA,ALJUNIED,2024-11,27,15.279143,412.536866
3,BAYSHORE,ALJUNIED,2024-11,37,7.242791,267.983271
4,FRANKEL,ALJUNIED,2024-11,46,4.101833,188.684336


In [601]:
# Inflow score aggregation by destination_subzone
inflow_score_train = inflow_train_score.groupby(['year_month','destination_subzone']).agg(inflow_train_score = ("trip_x_distance", "sum")).sort_values(by="inflow_train_score", ascending=False).reset_index()
inflow_score_train.head()

,year_month,destination_subzone,inflow_train_score
0,2025-01,JURONG WEST CENTRAL,2.349568e+07
1,2024-11,JURONG WEST CENTRAL,2.313269e+07
2,2024-12,JURONG WEST CENTRAL,2.205775e+07
3,2024-11,WOODLANDS REGIONAL CENTRE,1.899899e+07
4,2025-01,WOODLANDS REGIONAL CENTRE,1.898387e+07


In [602]:
# Outflow score aggregation by destination_subzone
outflow_score_train = inflow_train_score.groupby(['year_month','origin_subzone']).agg(outflow_train_score = ("trip_x_distance", "sum")).sort_values(by="outflow_train_score", ascending=False).reset_index()
outflow_score_train.head()

,year_month,origin_subzone,outflow_train_score
0,2025-01,JURONG WEST CENTRAL,2.360614e+07
1,2024-11,JURONG WEST CENTRAL,2.338940e+07
2,2024-12,JURONG WEST CENTRAL,2.241291e+07
3,2025-01,CHINATOWN,1.771731e+07
4,2025-01,TAMPINES EAST,1.741666e+07


## Total Train Tap In and Tap Out

In [603]:
train_pv_volume_grouped = (train_pv_volume.groupby(["PT_CODE", "YEAR_MONTH"])[['TOTAL_TAP_IN_VOLUME','TOTAL_TAP_OUT_VOLUME']]
                           .sum()
                           .reset_index()
                           .rename(columns={"PT_CODE": "station_code", "TOTAL_TAP_IN_VOLUME": "total_tap_in_volume", "TOTAL_TAP_OUT_VOLUME": "total_tap_out_volume", "YEAR_MONTH": "year_month"}))
train_pv_volume_grouped.head()

,station_code,year_month,total_tap_in_volume,total_tap_out_volume
0,BP10,2024-11,90783,93079
1,BP10,2024-12,86810,88179
2,BP10,2025-01,99878,99114
3,BP11,2024-11,106792,81538
4,BP11,2024-12,98661,74279


## Combine All Data

In [604]:
# Merge bus data
bus_data_com = (
    bus_stops_new
    .merge(bus_pv_volume_grouped, on="BusStopCode", how="left")
    .merge(bus_stop_hourly_buses, on="BusStopCode", how="left")
)

# Group bus data by subzone_name
bus_grouped = (
    bus_data_com.groupby(["subzone_name", "year_month"])
    .agg(
        total_bus_tap_in_volume=("total_tap_in_volume", "sum"),
        total_bus_tap_out_volume=("total_tap_out_volume", "sum"),
        total_buses_per_hour=("Buses_Per_Hour", "sum")
    )
    .reset_index()
)

# Join with inflow score
bus_combined = bus_grouped.merge(
    inflow_score_bus, 
    left_on=["subzone_name", "year_month"], 
    right_on=["destination_subzone", "year_month"],
    how="left"
)

# Join with outflow score
bus_combined = bus_combined.merge(
    outflow_score_bus, 
    left_on=["subzone_name", "year_month"], 
    right_on=["origin_subzone", "year_month"],
    how="left"
)

# Merge train data
mrt_data_com = (
    mrt
    .merge(train_pv_volume_grouped, on="station_code", how="left")
)

# Group train data by subzone_name
mrt_grouped = (
    mrt_data_com.groupby(["subzone_name", "year_month"])
    .agg(
        total_train_tap_in_volume=("total_tap_in_volume", "sum"),
        total_train_tap_out_volume=("total_tap_out_volume", "sum")
    )
    .reset_index()
)

# Join with inflow score
mrt_combined = mrt_grouped.merge(
    inflow_score_train, 
    left_on=["subzone_name", "year_month"], 
    right_on=["destination_subzone", "year_month"],
    how="left"
)

# Join with outflow score
mrt_combined = mrt_combined.merge(
    outflow_score_train, 
    left_on=["subzone_name", "year_month"], 
    right_on=["origin_subzone", "year_month"],
    how="left"
)

# Merge bus and train data at subzone level
final_combined_com = bus_combined.merge(mrt_combined, on=["subzone_name", "year_month"], how="outer")

# Display first few rows
final_combined_com.head()

,subzone_name,year_month,total_bus_tap_in_volume,total_bus_tap_out_volume,total_buses_per_hour,destination_subzone_x,inflow_bus_score,origin_subzone_x,outflow_bus_score,total_train_tap_in_volume,total_train_tap_out_volume,destination_subzone_y,inflow_train_score,origin_subzone_y,outflow_train_score
0,ADMIRALTY,2024-11,172382.0,212343.0,266.625,ADMIRALTY,409833.773100,ADMIRALTY,327012.685104,NaN,NaN,NaN,NaN,NaN,NaN
1,ADMIRALTY,2024-12,161893.0,206206.0,266.625,ADMIRALTY,410389.303520,ADMIRALTY,318889.492328,NaN,NaN,NaN,NaN,NaN,NaN
2,ADMIRALTY,2025-01,190118.0,233627.0,266.625,ADMIRALTY,436703.684905,ADMIRALTY,352333.068504,NaN,NaN,NaN,NaN,NaN,NaN
3,AIRPORT ROAD,2024-11,22289.0,22521.0,30.75,AIRPORT ROAD,66236.135966,AIRPORT ROAD,75450.600038,NaN,NaN,NaN,NaN,NaN,NaN
4,AIRPORT ROAD,2024-12,21756.0,21602.0,30.75,AIRPORT ROAD,63517.002541,AIRPORT ROAD,74337.864541,NaN,NaN,NaN,NaN,NaN,NaN


In [605]:
final_combined_com.sort_values(by=['total_train_tap_out_volume'], ascending=False)

,subzone_name,year_month,total_bus_tap_in_volume,total_bus_tap_out_volume,total_buses_per_hour,destination_subzone_x,inflow_bus_score,origin_subzone_x,outflow_bus_score,total_train_tap_in_volume,total_train_tap_out_volume,destination_subzone_y,inflow_train_score,origin_subzone_y,outflow_train_score
580,PATERSON,2024-12,170887.0,78148.0,80.875,PATERSON,3.185939e+05,PATERSON,5.629611e+05,2267663.0,2391999.0,PATERSON,1.602154e+07,PATERSON,1.543422e+07
149,CHINATOWN,2025-01,419814.0,332275.0,318.0,CHINATOWN,1.151725e+06,CHINATOWN,1.216227e+06,2370897.0,2384480.0,CHINATOWN,1.767262e+07,CHINATOWN,1.771731e+07
341,JURONG WEST CENTRAL,2025-01,2610091.0,2518721.0,1115.5,JURONG WEST CENTRAL,4.457866e+06,JURONG WEST CENTRAL,4.770483e+06,2210409.0,2237633.0,JURONG WEST CENTRAL,2.349568e+07,JURONG WEST CENTRAL,2.360614e+07
339,JURONG WEST CENTRAL,2024-11,2504790.0,2413880.0,1115.5,JURONG WEST CENTRAL,4.412568e+06,JURONG WEST CENTRAL,4.720205e+06,2175859.0,2194197.0,JURONG WEST CENTRAL,2.313269e+07,JURONG WEST CENTRAL,2.338940e+07
148,CHINATOWN,2024-12,382984.0,296447.0,318.0,CHINATOWN,1.014277e+06,CHINATOWN,1.088097e+06,2097280.0,2113364.0,CHINATOWN,1.576346e+07,CHINATOWN,1.548056e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
940,YUHUA WEST,2024-12,489789.0,437381.0,418.0,YUHUA WEST,1.080528e+06,YUHUA WEST,1.323778e+06,NaN,NaN,NaN,NaN,NaN,NaN
941,YUHUA WEST,2025-01,549307.0,493292.0,418.0,YUHUA WEST,1.189283e+06,YUHUA WEST,1.427444e+06,NaN,NaN,NaN,NaN,NaN,NaN
942,YUNNAN,2024-11,1000975.0,959439.0,856.25,YUNNAN,1.257700e+06,YUNNAN,1.622445e+06,NaN,NaN,NaN,NaN,NaN,NaN
943,YUNNAN,2024-12,945225.0,907994.0,856.25,YUNNAN,1.207856e+06,YUNNAN,1.529790e+06,NaN,NaN,NaN,NaN,NaN,NaN


In [606]:
# Drop unwanted columns
final_combined_com.drop(
    columns=["destination_subzone_x", "origin_subzone_x", "origin_subzone_y", "destination_subzone_y"],
    inplace=True
)

final_combined_com.fillna(0, inplace=True)
final_combined_com.head()

,subzone_name,year_month,total_bus_tap_in_volume,total_bus_tap_out_volume,total_buses_per_hour,inflow_bus_score,outflow_bus_score,total_train_tap_in_volume,total_train_tap_out_volume,inflow_train_score,outflow_train_score
0,ADMIRALTY,2024-11,172382.0,212343.0,266.625,409833.773100,327012.685104,0.0,0.0,0.0,0.0
1,ADMIRALTY,2024-12,161893.0,206206.0,266.625,410389.303520,318889.492328,0.0,0.0,0.0,0.0
2,ADMIRALTY,2025-01,190118.0,233627.0,266.625,436703.684905,352333.068504,0.0,0.0,0.0,0.0
3,AIRPORT ROAD,2024-11,22289.0,22521.0,30.75,66236.135966,75450.600038,0.0,0.0,0.0,0.0
4,AIRPORT ROAD,2024-12,21756.0,21602.0,30.75,63517.002541,74337.864541,0.0,0.0,0.0,0.0


In [607]:
area_df = pd.read_csv('../data/subzone_area.csv')
area_df

,subzone_name,area
0,DEPOT ROAD,0.445498
1,BUKIT MERAH,0.414702
2,CHINATOWN,0.591471
3,PHILLIP,0.039723
4,RAFFLES PLACE,0.190134
...,...,...
327,UPPER THOMSON,3.877598
328,SHANGRI-LA,0.692939
329,TOWNSVILLE,0.550385
330,MARYMOUNT,1.978472


### Divide columns by area to get number per square km

In [608]:
final_combined_com = final_combined_com.merge(area_df, on="subzone_name", how="left")
# List of columns to divide by 'area'
columns_to_normalize = [
    "total_bus_tap_in_volume", "total_bus_tap_out_volume", "total_buses_per_hour",
    "inflow_bus_score", "outflow_bus_score", "total_train_tap_in_volume",
    "total_train_tap_out_volume", "inflow_train_score", "outflow_train_score"
]

# Perform element-wise division by 'area', avoiding division by zero
final_combined_com[columns_to_normalize] = final_combined_com[columns_to_normalize].div(final_combined_com["area"], axis=0)

# Drop 'area' column if it's no longer needed
final_combined_com.drop(columns=["area"], inplace=True)
final_combined_com

,subzone_name,year_month,total_bus_tap_in_volume,total_bus_tap_out_volume,total_buses_per_hour,inflow_bus_score,outflow_bus_score,total_train_tap_in_volume,total_train_tap_out_volume,inflow_train_score,outflow_train_score
0,ADMIRALTY,2024-11,135631.516590,167073.146427,209.782652,3.224604e+05,2.572962e+05,0.0,0.0,0.0,0.0
1,ADMIRALTY,2024-12,127378.688699,162244.506445,209.782652,3.228975e+05,2.509048e+05,0.0,0.0,0.0,0.0
2,ADMIRALTY,2025-01,149586.341214,183819.565421,209.782652,3.436019e+05,2.772184e+05,0.0,0.0,0.0,0.0
3,AIRPORT ROAD,2024-11,45826.568849,46303.564855,63.222531,1.361826e+05,1.551277e+05,0.0,0.0,0.0,0.0
4,AIRPORT ROAD,2024-12,44730.711646,44414.084987,63.222531,1.305921e+05,1.528399e+05,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
940,YUHUA WEST,2024-12,626491.210846,559455.913242,534.665593,1.382108e+06,1.693251e+06,0.0,0.0,0.0,0.0
941,YUHUA WEST,2025-01,702620.939948,630971.913172,534.665593,1.521216e+06,1.825849e+06,0.0,0.0,0.0,0.0
942,YUNNAN,2024-11,450409.084161,431719.115161,385.287123,5.659276e+05,7.300520e+05,0.0,0.0,0.0,0.0
943,YUNNAN,2024-12,425323.236421,408570.389834,385.287123,5.434993e+05,6.883602e+05,0.0,0.0,0.0,0.0


In [609]:
# Run to download final_combined_com.csv
#final_combined_com.to_csv("../data/final_combined_com_v2.csv")

# **Accessibility Score Data**

## Tabulate mean frequency of buses & volume of passengers tapping in and out per subzone

In [610]:
# find the total volume of people travelling by bus and frequency per hour
bus_freq_acc = (
    bus_grouped.groupby(["subzone_name"])
    .agg(
        total_bus_tap_in_volume = ("total_bus_tap_in_volume", "mean"),
        total_bus_tap_out_volume = ("total_bus_tap_out_volume", "mean"),
        total_buses_per_hour = ("total_buses_per_hour", "mean")
    )
    .reset_index()
)
bus_freq_acc = bus_freq_acc.round({
    "total_bus_tap_in_volume": 3,
    "total_bus_tap_out_volume": 3,
    "total_buses_per_hour": 3
})

bus_freq_acc

,subzone_name,total_bus_tap_in_volume,total_bus_tap_out_volume,total_buses_per_hour
0,ADMIRALTY,174797.667,217392.000,266.625
1,AIRPORT ROAD,22416.333,22447.333,30.75
2,ALEXANDRA HILL,424159.000,518674.667,489.875
3,ALEXANDRA NORTH,49375.333,39977.333,81.125
4,ALJUNIED,1547271.333,1594564.000,1328.875
...,...,...,...,...
310,YISHUN SOUTH,808718.333,584999.000,557.0
311,YISHUN WEST,1181880.667,1059000.667,906.25
312,YUHUA EAST,455939.667,446377.667,490.875
313,YUHUA WEST,515424.667,462662.000,418.0


In [611]:
bus_stops_agg = bus_freq_acc.merge(bus_stops_agg, on = "subzone_name")
bus_stops_agg.head()

,subzone_name,total_bus_tap_in_volume,total_bus_tap_out_volume,total_buses_per_hour,subzone_code,planning_area_name,geog_boundary,num_bus_stops,num_bus_services,mean_bus_distance,AM_Peak_Max_Interval,AM_Peak_Min_Interval,AM_Offpeak_Min_Interval,AM_Offpeak_Max_Interval,PM_Peak_Min_Interval,PM_Peak_Max_Interval,PM_Offpeak_Min_Interval,PM_Offpeak_Max_Interval
0,ADMIRALTY,174797.667,217392.000,266.625,SBSZ05,SEMBAWANG,"POLYGON Z ((103.82855 1.45878 0, 103.82871 1.4...",21,75,17.162583,18.055929,13.936340,13.702140,19.793470,14.348735,19.291556,14.572823,19.082489
1,AIRPORT ROAD,22416.333,22447.333,30.75,PLSZ04,PAYA LEBAR,"POLYGON Z ((103.90133 1.35659 0, 103.90136 1.3...",3,11,11.011111,21.000000,10.555556,13.888889,22.361111,18.211111,25.931111,21.666667,27.920000
2,ALEXANDRA HILL,424159.000,518674.667,489.875,BMSZ07,BUKIT MERAH,"POLYGON Z ((103.81445 1.28547 0, 103.81433 1.2...",24,113,25.939027,15.959072,12.097402,11.356001,17.418553,12.084168,16.514496,12.712627,16.573347
3,ALEXANDRA NORTH,49375.333,39977.333,81.125,BMSZ06,BUKIT MERAH,"POLYGON Z ((103.8174 1.29433 0, 103.81787 1.29...",4,19,23.625000,17.121667,11.375000,10.666667,16.708333,11.500000,15.833333,14.708333,19.333333
4,ALJUNIED,1547271.333,1594564.000,1328.875,GLSZ04,GEYLANG,"POLYGON Z ((103.89127 1.32132 0, 103.89153 1.3...",45,328,24.254981,14.536552,11.733150,11.426836,17.735702,12.691759,16.684773,14.930621,18.699993


## MRT Stations

### Mean train tap in and out volume

In [612]:
mrt_freq_acc = (
    mrt_grouped.groupby(["subzone_name"])
    .agg(
        total_train_tap_in_volume=("total_train_tap_in_volume", "mean"),
        total_train_tap_out_volume=("total_train_tap_out_volume", "mean")
    )
    .reset_index()
)
mrt_freq_acc = mrt_freq_acc.round({
    "total_train_tap_in_volume": 3,
    "total_train_tap_out_volume": 3
})

mrt_freq_acc.head()

,subzone_name,total_train_tap_in_volume,total_train_tap_out_volume
0,ALJUNIED,1067093.000,1070251.333
1,ANAK BUKIT,419043.333,414933.333
2,ANCHORVALE,292932.667,284416.333
3,BANGKIT,117476.667,114693.667
4,BAYFRONT SUBZONE,1180836.333,1118269.000


## MRT Proximities
- Generate random point in each region in SG 
- Query information about the route from the random points to each MRT/LRT station with OneMap API

### Create Region Dataframe with updated boundaries
- some areas are multipolygon types (e.g. Semakau) and they usually comprise of multple areas. Some areas are unreachable through public transport, so filter out only the polygonal types

In [613]:
region_df = subzone[subzone["geog_boundary"].geom_type == "Polygon"].dissolve(by = "region_code").reset_index().loc[:, ["region_code","region_name","geog_boundary"]]
region_df.head()

,region_code,region_name,geog_boundary
0,CR,CENTRAL REGION,"POLYGON Z ((103.81454 1.28239 0, 103.81774 1.2..."
1,ER,EAST REGION,"POLYGON Z ((103.93208 1.30555 0, 103.93208 1.3..."
2,NER,NORTH-EAST REGION,"POLYGON Z ((103.84853 1.36879 0, 103.84834 1.3..."
3,NR,NORTH REGION,"POLYGON Z ((103.85563 1.42734 0, 103.85955 1.4..."
4,WR,WEST REGION,"POLYGON Z ((103.76545 1.30256 0, 103.76698 1.2..."


### Generate a Random Point for Each Region in SG

In [614]:
def generate_random_point(polygon):
    minx, miny, maxx, maxy = polygon.bounds
    rand_point = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
    return rand_point
# generate a random point for each region in sg
region_df['random_point']= region_df.apply(lambda region: generate_random_point(region["geog_boundary"]), axis=1)
region_df

,region_code,region_name,geog_boundary,random_point
0,CR,CENTRAL REGION,"POLYGON Z ((103.81454 1.28239 0, 103.81774 1.2...",POINT (103.81112 1.28028)
1,ER,EAST REGION,"POLYGON Z ((103.93208 1.30555 0, 103.93208 1.3...",POINT (103.92825 1.30627)
2,NER,NORTH-EAST REGION,"POLYGON Z ((103.84853 1.36879 0, 103.84834 1.3...",POINT (103.84581 1.37029)
3,NR,NORTH REGION,"POLYGON Z ((103.85563 1.42734 0, 103.85955 1.4...",POINT (103.83983 1.42372)
4,WR,WEST REGION,"POLYGON Z ((103.76545 1.30256 0, 103.76698 1.2...",POINT (103.76742 1.30692)


### Query Routes

In [627]:
# Run this to download mrt_proximities.csv
def query_route_info(mrt_df, region_df):
    all_routes =[]
    for _, station in mrt_df.iterrows():
        mrt_lat = station.geometry.y
        mrt_lng = station.geometry.x
        route_info = {}
        for _, region in region_df.iterrows():
            region_name = region["region_name"]
            pt = region["random_point"]
            lng = pt.x
            lat = pt.y

            curr_dt = datetime.today().strftime('%m-%d-%Y')

            url = f"https://www.onemap.gov.sg/api/public/routingsvc/route?start={lat}%2C{lng}&end={mrt_lat}%2C{mrt_lng}&routeType=pt&date={curr_dt}&time=12%3A00%3A00&mode=TRANSIT&maxWalkDistance=1000&numItineraries=1"
            headers = {
                "Authorization": ONEMAP_ACCOUNT_KEY
            }

            response = requests.request("GET", url, headers=headers)

            if response.status_code == 200:
                itinerary = response.json()["plan"].get("itineraries", [])
                if itinerary:
                    itinerary = itinerary[0]
                    duration = itinerary.get("duration")
                    walkDistance = itinerary.get("walkDistance")
                    fare = itinerary.get("fare")
                    route_info.update({
                        "mrt_station" :station["station_name"],
                        f"duration_from_{region_name}": duration,
                        f"walking_dist_from_{region_name}": walkDistance,
                        f"fare_from_{region_name}": fare})

            else:
                route_info.update({
                "mrt_station" : station["station_name"],
                f"duration_from_{region_name}": None,
                f"walking_dist_from_{region_name}": None,
                f"fare_from_{region_name}": None})
            time.sleep(0.25)
        all_routes.append(route_info)
    return all_routes

""" # Uncomment and run this to download mrt_proximities.csv
mrt_prx = pd.DataFrame(query_route_info(mrt, region_df))
mrt_prx.head()
mrt_prx.to_csv("../data/mrt_proximities.csv", index = False) """

### Cleaning of MRT and LRT Routes Data

In [633]:
# Run this to read the mrt_proximities dataframe
mrt_routes = pd.read_csv("../data/mrt_proximities.csv")
mrt_routes.head()

,mrt_station,duration_from_CENTRAL REGION,walking_dist_from_CENTRAL REGION,fare_from_CENTRAL REGION,duration_from_EAST REGION,walking_dist_from_EAST REGION,fare_from_EAST REGION,duration_from_NORTH-EAST REGION,walking_dist_from_NORTH-EAST REGION,fare_from_NORTH-EAST REGION,duration_from_NORTH REGION,walking_dist_from_NORTH REGION,fare_from_NORTH REGION,duration_from_WEST REGION,walking_dist_from_WEST REGION,fare_from_WEST REGION
0,Choa Chu Kang,2426,308.93,2.10,4284,345.27,2.36,2773,563.28,2.23,3199,481.13,2.23,2568,839.33,1.89
1,Fajar,3988,317.66,2.10,5534,547.50,info unavailable,3938,1009.41,2.20,4364,927.26,2.32,3890,1235.24,1.81
2,Segar,4478,518.25,2.14,5694,917.68,info unavailable,4156,1306.59,2.20,4582,1224.44,2.32,4420,1521.66,1.89
3,Jelapang,4327,559.26,2.10,4837,389.90,2.33,3369,479.15,2.17,3795,397.00,2.32,3805,694.22,1.89
4,Senja,4144,276.71,2.06,4836,486.75,2.33,3504,519.56,2.26,3975,437.41,2.20,3896,801.64,1.89


In [634]:
mrt_routes = mrt_routes.replace("info unavailable", np.nan) # For cells with no information available, convert to NA

# Convert fare columns to numeric
fare_columns = ['fare_from_CENTRAL REGION', 'fare_from_EAST REGION','fare_from_NORTH REGION', 'fare_from_NORTH-EAST REGION', 'fare_from_WEST REGION']
mrt_routes[fare_columns] = mrt_routes[fare_columns].apply(pd.to_numeric, errors='coerce') 

for col in mrt_routes.columns[1:]:
        upper_pct = np.nanpercentile(mrt_routes[col].dropna(), 99) #Fill all NA values with 99th Percentile
        mrt_routes[col].fillna(upper_pct, inplace = True)

/var/folders/2m/8llkplds47v3b1fv6f6vqv0m0000gn/T/ipykernel_11438/3955613696.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  mrt_routes[col].fillna(upper_pct, inplace = True)


In [635]:
# Merge mrt routes data with main mrt data to obtain subzone name
mrt_routes = mrt.loc[:,["station_name", "subzone_name"]].merge(mrt_routes, left_on = "station_name", right_on = "mrt_station", how = "left").drop_duplicates()

#Aggregate data on a subzone level
mrt_routes_final = mrt_routes.drop(columns=['station_name', 'mrt_station']).groupby('subzone_name').mean().reset_index()
mrt_routes_final = mrt_routes_final.rename(columns={col: col.lower() + "_to_mrt" for col in mrt_routes_final.columns if col != "subzone_name"})
mrt_routes_final.head()

,subzone_name,duration_from_central region_to_mrt,walking_dist_from_central region_to_mrt,fare_from_central region_to_mrt,duration_from_east region_to_mrt,walking_dist_from_east region_to_mrt,fare_from_east region_to_mrt,duration_from_north-east region_to_mrt,walking_dist_from_north-east region_to_mrt,fare_from_north-east region_to_mrt,duration_from_north region_to_mrt,walking_dist_from_north region_to_mrt,fare_from_north region_to_mrt,duration_from_west region_to_mrt,walking_dist_from_west region_to_mrt,fare_from_west region_to_mrt
0,ALJUNIED,2906.000000,624.246667,1.92,2057.666667,559.003333,1.683333,2465.333333,1133.783333,1.836667,4263.000000,1240.456667,2.25,3973.666667,1229.46,2.150000
1,ANAK BUKIT,2803.000000,142.420000,1.81,3736.000000,208.110000,2.200000,2219.000000,642.960000,1.930000,4109.000000,560.810000,2.34,2264.000000,667.35,1.500000
2,ANCHORVALE,3818.333333,694.786667,2.23,3859.666667,183.030000,2.100000,2319.000000,317.643333,1.680000,2533.333333,1069.853333,1.66,4632.666667,1368.76,2.293333
3,BANGKIT,3778.500000,73.335000,2.10,4834.000000,270.550000,2.315000,3382.500000,472.190000,2.185000,3808.500000,390.040000,2.32,3416.000000,687.26,1.810000
4,BAYFRONT SUBZONE,2449.000000,458.780000,1.77,2657.000000,867.460000,1.890000,2597.000000,689.310000,1.890000,4467.000000,781.320000,2.33,3371.000000,579.56,1.930000


## Aggregate MRT Station data

In [636]:
# Create column for mrt line
mrt_acc = mrt
mrt_acc["line"] = mrt_acc["station_code"].str[:2]

# Create column for rail_type: MRT, LRT
mrt_acc["rail_type"] = np.where(
    mrt_acc["line"].isin(["BP", "PE", "PT", "PW", "SE", "ST", "SW"]),
    "LRT",
    "MRT"
)
mrt_acc.head()

,station_code,station_name,lat,lon,geometry,index_right,subzone_no,subzone_name,subzone_code,planning_area_name,planning_area_code,region_name,region_code,geog_boundary,area,line,rail_type
0,BP1,Choa Chu Kang,1.384755,103.744538,POINT (103.74454 1.38476),244,3,CHOA CHU KANG CENTRAL,CKSZ03,CHOA CHU KANG,CK,WEST REGION,WR,"POLYGON Z ((103.75265 1.38286 0, 103.7526 1.38...",1.103706,BP,LRT
1,BP10,Fajar,1.384573,103.770887,POINT (103.77089 1.38457),245,3,FAJAR,BPSZ03,BUKIT PANJANG,BP,WEST REGION,WR,"POLYGON Z ((103.77499 1.3845 0, 103.77465 1.38...",0.673347,BP,LRT
2,BP11,Segar,1.387785,103.769600,POINT (103.7696 1.38779),247,2,SAUJANA,BPSZ02,BUKIT PANJANG,BP,WEST REGION,WR,"POLYGON Z ((103.76712 1.38215 0, 103.76708 1.3...",0.679933,BP,LRT
3,BP12,Jelapang,1.386739,103.764534,POINT (103.76453 1.38674),247,2,SAUJANA,BPSZ02,BUKIT PANJANG,BP,WEST REGION,WR,"POLYGON Z ((103.76712 1.38215 0, 103.76708 1.3...",0.679933,BP,LRT
4,BP13,Senja,1.382725,103.762344,POINT (103.76234 1.38273),246,1,SENJA,BPSZ01,BUKIT PANJANG,BP,WEST REGION,WR,"POLYGON Z ((103.76441 1.38887 0, 103.76435 1.3...",0.629451,BP,LRT


### Aggregate MRT/LRT data on Subzone Level
- Number of unique MRT/LRT stations
- Number of train lines accessible within subzone
- Number of rail types accessible within subzone

In [637]:
mrt_agg_acc = mrt_acc.groupby(["subzone_code", "subzone_name"]).agg(
    num_stations_per_subzone=("station_code", "nunique"),
    num_train_lines_per_subzone=("line", "nunique"),
    num_rail_types_per_subzone=("rail_type","nunique")
).reset_index()

mrt_agg_acc = mrt_agg_acc.merge(mrt_freq_acc, on = "subzone_name")
mrt_agg_acc.head()

,subzone_code,subzone_name,num_stations_per_subzone,num_train_lines_per_subzone,num_rail_types_per_subzone,total_train_tap_in_volume,total_train_tap_out_volume
0,AMSZ02,CHENG SAN,1,1,1,1202331.333,1324377.000
1,AMSZ06,KEBUN BAHRU,1,1,1,314766.333,298753.333
2,AMSZ08,TAGORE,1,1,1,142141.333,141936.000
3,AMSZ10,YIO CHU KANG,1,1,1,588218.667,535445.333
4,BDSZ01,KAKI BUKIT,2,1,1,544531.000,511861.000


## Private Transport

In [638]:
region_df = gpd.GeoDataFrame(region_df, geometry = "geog_boundary")

In [639]:
region_df['centroids']= gdf.geometry.centroid
region_df.set_crs("EPSG:4326", allow_override=True)
region_df.head()

/var/folders/2m/8llkplds47v3b1fv6f6vqv0m0000gn/T/ipykernel_11438/488140810.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  region_df['centroids']= gdf.geometry.centroid


,region_code,region_name,geog_boundary,random_point,centroids
0,CR,CENTRAL REGION,"POLYGON Z ((103.81454 1.28239 0, 103.81774 1.2...",POINT (103.81112 1.28028),POINT (103.80856 1.28222)
1,ER,EAST REGION,"POLYGON Z ((103.93208 1.30555 0, 103.93208 1.3...",POINT (103.92825 1.30627),POINT (103.81859 1.28201)
2,NER,NORTH-EAST REGION,"POLYGON Z ((103.84853 1.36879 0, 103.84834 1.3...",POINT (103.84581 1.37029),POINT (103.84369 1.27997)
3,NR,NORTH REGION,"POLYGON Z ((103.85563 1.42734 0, 103.85955 1.4...",POINT (103.83983 1.42372),POINT (103.84865 1.28528)
4,WR,WEST REGION,"POLYGON Z ((103.76545 1.30256 0, 103.76698 1.2...",POINT (103.76742 1.30692),POINT (103.85101 1.28372)


In [623]:
subzone['centroids']= gdf.geometry.centroid
subzone.set_crs("EPSG:4326", allow_override=True)
subzone.head()

/var/folders/2m/8llkplds47v3b1fv6f6vqv0m0000gn/T/ipykernel_11438/1180706107.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  subzone['centroids']= gdf.geometry.centroid


,subzone_no,subzone_name,subzone_code,planning_area_name,planning_area_code,region_name,region_code,geometry,geog_boundary,area,centroids
0,12,DEPOT ROAD,BMSZ12,BUKIT MERAH,BM,CENTRAL REGION,CR,"POLYGON Z ((25910.344 29425.34 0, 26265.823 29...","POLYGON Z ((103.81454 1.28239 0, 103.81774 1.2...",0.445498,POINT (103.80856 1.28222)
1,2,BUKIT MERAH,BMSZ02,BUKIT MERAH,BM,CENTRAL REGION,CR,"POLYGON Z ((26750.092 29216.098 0, 26751.912 2...","POLYGON Z ((103.82209 1.28049 0, 103.8221 1.28...",0.414702,POINT (103.81859 1.28201)
2,3,CHINATOWN,OTSZ03,OUTRAM,OT,CENTRAL REGION,CR,"POLYGON Z ((29161.201 29723.071 0, 29189.033 2...","POLYGON Z ((103.84375 1.28508 0, 103.844 1.284...",0.591471,POINT (103.84369 1.27997)
3,4,PHILLIP,DTSZ04,DOWNTOWN CORE,DT,CENTRAL REGION,CR,"POLYGON Z ((29814.107 29616.894 0, 29806.682 2...","POLYGON Z ((103.84962 1.28412 0, 103.84955 1.2...",0.039723,POINT (103.84865 1.28528)
4,5,RAFFLES PLACE,DTSZ05,DOWNTOWN CORE,DT,CENTRAL REGION,CR,"POLYGON Z ((30137.768 29843.194 0, 30138.417 2...","POLYGON Z ((103.85253 1.28617 0, 103.85253 1.2...",0.190134,POINT (103.85101 1.28372)


In [640]:
# Use OneMap API to query the route
def query_route_priv(origin_df, dest_df, mode):
    all_routes =[]
    for _, origin in origin_df.iterrows():
        origin_lat = origin.centroids.y
        origin_lng = origin.centroids.x
        origin_code = origin.region_code
        route_info = {}
        for _, dest in dest_df.iterrows():
            dest_code = dest.subzone_code
            dest_lng = dest.centroids.x
            dest_lat = dest.centroids.y

            url = f"https://www.onemap.gov.sg/api/public/routingsvc/route?start={origin_lat}%2C{origin_lng}&end={dest_lat}%2C{dest_lng}&routeType={mode}&mode=TRANSIT"
            
            headers = {
                "Authorization": ONEMAP_ACCOUNT_KEY
            }

            response = requests.request("GET", url, headers=headers)
            print(response.text)

            route_info = {}
            if response.status_code == 200:
                route = response.json()["route_summary"]
                total_time = route["total_time"]
                total_distance = route["total_distance"]
                if route:
                    route_info.update({
                        "subzone_code" :dest_code,
                        "region_code": origin_code,
                        "total_time": total_time,
                        "total_distance": total_distance})
            else:
                route_info.update({
                        "subzone_code" :dest_code,
                        "region_code": origin_code,
                        "total_time": None,
                        "total_distance": None})
            time.sleep(0.25)
            all_routes.append(route_info)
    return all_routes

In [641]:
def query_route_public(origin_df, dest_df):
    all_routes =[]
    for _, origin in origin_df.iterrows():
        origin_lat = origin.centroids.y
        origin_lng = origin.centroids.x
        origin_code = origin.region_code
        route_info = {}
        for _, dest in dest_df.iterrows():
            dest_code = dest.subzone_code
            dest_lng = dest.centroids.x
            dest_lat = dest.centroids.y

            curr_dt = datetime.today().strftime('%m-%d-%Y')

            url = f"https://www.onemap.gov.sg/api/public/routingsvc/route?start={origin_lat}%2C{origin_lng}&end={dest_lat}%2C{dest_lng}&routeType=pt&date={curr_dt}&time=12%3A00%3A00&mode=TRANSIT&maxWalkDistance=1000&numItineraries=1"
            headers = {
                "Authorization": ONEMAP_ACCOUNT_KEY
            }

            response = requests.request("GET", url, headers=headers)
            #print(response.text)

            route_info = {}
            if response.status_code == 200:
                itinerary = response.json()["plan"].get("itineraries", [])
                if itinerary:
                    itinerary = itinerary[0]
                    duration = itinerary.get("duration")
                    route_info.update({
                        "subzone_code" :dest_code,
                        "region_code": origin_code,
                        "total_time": duration})
            else:
                route_info.update({
                        "subzone_code" :dest_code,
                        "region_code": origin_code,
                        "total_time": None})
            time.sleep(0.25)
            all_routes.append(route_info)
            #print(all_routes)
    return all_routes

In [642]:
""" #Uncomment and run this to download
route_private = pd.DataFrame(query_route_priv(region_df, subzone, "drive"))
route_private.head()
route_private.to_csv("../data/route_private.csv")
route_pt = pd.DataFrame(query_route_public(region_df, subzone))
route_pt.head()
route_pt.to_csv("../data/route_pt.csv") """

' #Uncomment and run this to download\nroute_private = pd.DataFrame(query_route_priv(region_df, subzone, "drive"))\nroute_private.head()\nroute_private.to_csv("../data/route_private.csv")\nroute_pt = pd.DataFrame(query_route_public(region_df, subzone))\nroute_pt.head()\nroute_pt.to_csv("../data/route_pt.csv") '

In [643]:
# Run this to download the route information for public and private transport from the centroid of each subzone to the centroid of each region
route_priv = pd.read_csv("../data/route_private.csv", index_col = 0)
route_pt = pd.read_csv("../data/route_pt.csv", index_col = 0)

In [696]:
subzone[subzone['subzone_code']=='DTSZ07']

,subzone_no,subzone_name,subzone_code,planning_area_name,planning_area_code,region_name,region_code,geometry,geog_boundary,area,centroids
39,7,MAXWELL,DTSZ07,DOWNTOWN CORE,DT,CENTRAL REGION,CR,"POLYGON Z ((29341.726 29189.812 0, 29343.287 2...","POLYGON Z ((103.84537 1.28026 0, 103.84539 1.2...",0.064126,POINT (103.84583 1.27942)


In [692]:
region_df

,region_code,region_name,geog_boundary,random_point,centroids
0,CR,CENTRAL REGION,"POLYGON Z ((103.81454 1.28239 0, 103.81774 1.2...",POINT (103.81112 1.28028),POINT (103.80856 1.28222)
1,ER,EAST REGION,"POLYGON Z ((103.93208 1.30555 0, 103.93208 1.3...",POINT (103.92825 1.30627),POINT (103.81859 1.28201)
2,NER,NORTH-EAST REGION,"POLYGON Z ((103.84853 1.36879 0, 103.84834 1.3...",POINT (103.84581 1.37029),POINT (103.84369 1.27997)
3,NR,NORTH REGION,"POLYGON Z ((103.85563 1.42734 0, 103.85955 1.4...",POINT (103.83983 1.42372),POINT (103.84865 1.28528)
4,WR,WEST REGION,"POLYGON Z ((103.76545 1.30256 0, 103.76698 1.2...",POINT (103.76742 1.30692),POINT (103.85101 1.28372)


In [680]:
route_priv[route_priv['total_time']==0]

,subzone_code,region_code,total_time,total_distance
0,BMSZ12,CR,0.0,0.0
333,BMSZ02,ER,0.0,0.0


In [695]:
route_pt.sort_values(by='total_time')

,subzone_code,region_code,total_time
703,DTSZ07,NER,261.0
1415,DTSZ02,WR,339.0
1014,SRSZ03,NR,348.0
1331,DTSZ04,WR,377.0
1000,DTSZ05,NR,377.0
...,...,...,...
307,SLSZ02,CR,7323.0
1293,CHSZ03,NR,7443.0
961,CHSZ03,NER,7511.0
629,CHSZ03,ER,8107.0


In [644]:
for col in route_priv.columns[2:]:
        upper_pct = np.nanpercentile(route_priv[col].dropna(), 99) #Fill all NA values with 99th Percentile
        route_priv[col].fillna(upper_pct, inplace = True)

for col in route_pt.columns[2:]:
        upper_pct = np.nanpercentile(route_pt[col].dropna(), 99) #Fill all NA values with 99th Percentile
        route_pt[col].fillna(upper_pct, inplace = True)

/var/folders/2m/8llkplds47v3b1fv6f6vqv0m0000gn/T/ipykernel_11438/1912551857.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  route_priv[col].fillna(upper_pct, inplace = True)
/var/folders/2m/8llkplds47v3b1fv6f6vqv0m0000gn/T/ipykernel_11438/1912551857.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alw

In [645]:
route_priv.isna().sum()

subzone_code      0
region_code       0
total_time        0
total_distance    0
dtype: int64

In [646]:
route_pt.isna().sum()

subzone_code    0
region_code     0
total_time      0
dtype: int64

In [647]:
route_priv.head()

,subzone_code,region_code,total_time,total_distance
0,BMSZ12,CR,0.0,0.0
1,BMSZ02,CR,220.0,1461.0
2,OTSZ03,CR,702.0,5475.0
3,DTSZ04,CR,632.0,5529.0
4,DTSZ05,CR,718.0,6180.0


In [648]:
#Combine both private and public
route_total = route_priv.merge(route_pt, on = ["subzone_code", "region_code"], suffixes= ("_priv", "_public"))
route_total["priv_to_public_ratio"] = route_total["total_time_priv"]/route_total["total_time_public"]

In [649]:
route_total.describe()

,total_time_priv,total_distance,total_time_public,priv_to_public_ratio
count,1660.000000,1660.000000,1660.000000,1660.000000
mean,1306.367530,14309.463554,2956.631205,0.444179
std,645.492539,8165.123873,1282.028904,0.186019
min,0.000000,0.000000,261.000000,0.000000
25%,803.000000,6906.500000,1963.750000,0.365773
50%,1288.000000,14317.500000,2896.000000,0.436958
75%,1776.750000,20498.000000,3757.750000,0.515504
max,3796.000000,41621.000000,8603.000000,4.884291


In [650]:
# Find the mean ratio per subzone, aggregating across the different regions
route_total_agg = route_total.groupby(["subzone_code"])["priv_to_public_ratio"].mean().reset_index()

## Combine All Data

In [651]:
final_combined_acc = bus_stops_agg.merge(mrt_agg_acc.drop(columns = "subzone_name"), on = "subzone_code", how = "left")
#for subzones without bus stops or mrt stations, impute 0 for columns
final_combined_acc.fillna(0, inplace = True)

final_combined_acc = final_combined_acc.merge(mrt_routes_final, on = "subzone_name", how = "left")
#for subzones without mrt stations, impute an improbably high numeric value in the mrt info columns (since a lower value suggests that the subzone is more accessible)

for col in final_combined_acc.columns[23:]:
        upper_pct = np.nanpercentile(final_combined_acc[col].dropna(), 99) #Fill all NA values with 99th Percentile
        final_combined_acc[col].fillna(upper_pct, inplace = True)

final_combined_acc = final_combined_acc.merge(subzone.loc[:,['subzone_code','area']], on = "subzone_code", how = "left")
final_combined_acc.head()

/var/folders/2m/8llkplds47v3b1fv6f6vqv0m0000gn/T/ipykernel_11438/2625404277.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_combined_acc[col].fillna(upper_pct, inplace = True)


,subzone_name,total_bus_tap_in_volume,total_bus_tap_out_volume,total_buses_per_hour,subzone_code,planning_area_name,geog_boundary,num_bus_stops,num_bus_services,mean_bus_distance,...,duration_from_north-east region_to_mrt,walking_dist_from_north-east region_to_mrt,fare_from_north-east region_to_mrt,duration_from_north region_to_mrt,walking_dist_from_north region_to_mrt,fare_from_north region_to_mrt,duration_from_west region_to_mrt,walking_dist_from_west region_to_mrt,fare_from_west region_to_mrt,area
0,ADMIRALTY,174797.667,217392.000,266.625,SBSZ05,SEMBAWANG,"POLYGON Z ((103.82855 1.45878 0, 103.82871 1.4...",21,75,17.162583,...,4866.400000,1791.491000,2.360000,6217.2,1662.159000,2.47,5385.108333,2116.052,2.38973,1.270958
1,AIRPORT ROAD,22416.333,22447.333,30.75,PLSZ04,PAYA LEBAR,"POLYGON Z ((103.90133 1.35659 0, 103.90136 1.3...",3,11,11.011111,...,4866.400000,1791.491000,2.360000,6217.2,1662.159000,2.47,5385.108333,2116.052,2.38973,0.486377
2,ALEXANDRA HILL,424159.000,518674.667,489.875,BMSZ07,BUKIT MERAH,"POLYGON Z ((103.81445 1.28547 0, 103.81433 1.2...",24,113,25.939027,...,4866.400000,1791.491000,2.360000,6217.2,1662.159000,2.47,5385.108333,2116.052,2.38973,1.037840
3,ALEXANDRA NORTH,49375.333,39977.333,81.125,BMSZ06,BUKIT MERAH,"POLYGON Z ((103.8174 1.29433 0, 103.81787 1.29...",4,19,23.625000,...,4866.400000,1791.491000,2.360000,6217.2,1662.159000,2.47,5385.108333,2116.052,2.38973,0.295823
4,ALJUNIED,1547271.333,1594564.000,1328.875,GLSZ04,GEYLANG,"POLYGON Z ((103.89127 1.32132 0, 103.89153 1.3...",45,328,24.254981,...,2465.333333,1133.783333,1.836667,4263.0,1240.456667,2.25,3973.666667,1229.460,2.15000,2.982971


In [652]:
final_combined_acc = final_combined_acc.merge(route_total_agg, on = "subzone_code")
final_combined_acc.head()

,subzone_name,total_bus_tap_in_volume,total_bus_tap_out_volume,total_buses_per_hour,subzone_code,planning_area_name,geog_boundary,num_bus_stops,num_bus_services,mean_bus_distance,...,walking_dist_from_north-east region_to_mrt,fare_from_north-east region_to_mrt,duration_from_north region_to_mrt,walking_dist_from_north region_to_mrt,fare_from_north region_to_mrt,duration_from_west region_to_mrt,walking_dist_from_west region_to_mrt,fare_from_west region_to_mrt,area,priv_to_public_ratio
0,ADMIRALTY,174797.667,217392.000,266.625,SBSZ05,SEMBAWANG,"POLYGON Z ((103.82855 1.45878 0, 103.82871 1.4...",21,75,17.162583,...,1791.491000,2.360000,6217.2,1662.159000,2.47,5385.108333,2116.052,2.38973,1.270958,0.623183
1,AIRPORT ROAD,22416.333,22447.333,30.75,PLSZ04,PAYA LEBAR,"POLYGON Z ((103.90133 1.35659 0, 103.90136 1.3...",3,11,11.011111,...,1791.491000,2.360000,6217.2,1662.159000,2.47,5385.108333,2116.052,2.38973,0.486377,0.438736
2,ALEXANDRA HILL,424159.000,518674.667,489.875,BMSZ07,BUKIT MERAH,"POLYGON Z ((103.81445 1.28547 0, 103.81433 1.2...",24,113,25.939027,...,1791.491000,2.360000,6217.2,1662.159000,2.47,5385.108333,2116.052,2.38973,1.037840,0.313048
3,ALEXANDRA NORTH,49375.333,39977.333,81.125,BMSZ06,BUKIT MERAH,"POLYGON Z ((103.8174 1.29433 0, 103.81787 1.29...",4,19,23.625000,...,1791.491000,2.360000,6217.2,1662.159000,2.47,5385.108333,2116.052,2.38973,0.295823,0.258394
4,ALJUNIED,1547271.333,1594564.000,1328.875,GLSZ04,GEYLANG,"POLYGON Z ((103.89127 1.32132 0, 103.89153 1.3...",45,328,24.254981,...,1133.783333,1.836667,4263.0,1240.456667,2.25,3973.666667,1229.460,2.15000,2.982971,0.518057


In [653]:
# List of columns to divide by 'area'
columns_to_normalize_acc = [
    "num_bus_stops","num_bus_services",
    "num_stations_per_subzone", "num_train_lines_per_subzone",
    "num_rail_types_per_subzone","total_bus_tap_in_volume","total_bus_tap_out_volume",
    "total_buses_per_hour","total_train_tap_in_volume","total_train_tap_out_volume"
]

# Perform element-wise division by 'area', avoiding division by zero
final_combined_acc[columns_to_normalize_acc] = final_combined_acc[columns_to_normalize_acc].div(final_combined_acc["area"], axis=0)

final_combined_acc.head()

,subzone_name,total_bus_tap_in_volume,total_bus_tap_out_volume,total_buses_per_hour,subzone_code,planning_area_name,geog_boundary,num_bus_stops,num_bus_services,mean_bus_distance,...,walking_dist_from_north-east region_to_mrt,fare_from_north-east region_to_mrt,duration_from_north region_to_mrt,walking_dist_from_north region_to_mrt,fare_from_north region_to_mrt,duration_from_west region_to_mrt,walking_dist_from_west region_to_mrt,fare_from_west region_to_mrt,area,priv_to_public_ratio
0,ADMIRALTY,137532.182430,171045.739431,209.782652,SBSZ05,SEMBAWANG,"POLYGON Z ((103.82855 1.45878 0, 103.82871 1.4...",16.522966,59.010591,17.162583,...,1791.491000,2.360000,6217.2,1662.159000,2.47,5385.108333,2116.052,2.38973,1.270958,0.623183
1,AIRPORT ROAD,46088.367695,46152.104231,63.222531,PLSZ04,PAYA LEBAR,"POLYGON Z ((103.90133 1.35659 0, 103.90136 1.3...",6.168052,22.616190,11.011111,...,1791.491000,2.360000,6217.2,1662.159000,2.47,5385.108333,2116.052,2.38973,0.486377,0.438736
2,ALEXANDRA HILL,408694.021923,499763.616300,472.013995,BMSZ07,BUKIT MERAH,"POLYGON Z ((103.81445 1.28547 0, 103.81433 1.2...",23.124952,108.879982,25.939027,...,1791.491000,2.360000,6217.2,1662.159000,2.47,5385.108333,2116.052,2.38973,1.037840,0.313048
3,ALEXANDRA NORTH,166908.374821,135139.375783,274.234949,BMSZ06,BUKIT MERAH,"POLYGON Z ((103.8174 1.29433 0, 103.81787 1.29...",13.521600,64.227600,23.625000,...,1791.491000,2.360000,6217.2,1662.159000,2.47,5385.108333,2116.052,2.38973,0.295823,0.258394
4,ALJUNIED,518701.385040,534555.599716,445.487025,GLSZ04,GEYLANG,"POLYGON Z ((103.89127 1.32132 0, 103.89153 1.3...",15.085630,109.957478,24.254981,...,1133.783333,1.836667,4263.0,1240.456667,2.25,3973.666667,1229.460,2.15000,2.982971,0.518057


In [654]:
desired_order = [
    #regional infomation
    'subzone_name', 'subzone_code', 'geog_boundary', 'planning_area_name', 'area', "priv_to_public_ratio",

    # Bus information
    'total_bus_tap_in_volume', 'total_bus_tap_out_volume',
    'total_buses_per_hour', 'num_bus_stops', 'num_bus_services',
    'mean_bus_distance',
    'AM_Peak_Min_Interval','AM_Peak_Max_Interval',
    'AM_Offpeak_Min_Interval','AM_Offpeak_Max_Interval',
    'PM_Peak_Min_Interval','PM_Peak_Max_Interval',
    'PM_Offpeak_Min_Interval','PM_Offpeak_Max_Interval',

    # MRT/LRT
    'num_stations_per_subzone', 'num_train_lines_per_subzone', 'num_rail_types_per_subzone', 
    'total_train_tap_in_volume', 'total_train_tap_out_volume',

    # Routes from each region to each mrt station
    'duration_from_central region_to_mrt',
    'walking_dist_from_central region_to_mrt',
    'fare_from_central region_to_mrt', 'duration_from_east region_to_mrt',
    'walking_dist_from_east region_to_mrt', 'fare_from_east region_to_mrt',
    'duration_from_north-east region_to_mrt',
    'walking_dist_from_north-east region_to_mrt',
    'fare_from_north-east region_to_mrt',
    'duration_from_north region_to_mrt',
    'walking_dist_from_north region_to_mrt', 'fare_from_north region_to_mrt',
    'duration_from_west region_to_mrt',
    'walking_dist_from_west region_to_mrt', 'fare_from_west region_to_mrt'

]

# Reorder the DataFrame columns
final_combined_acc = final_combined_acc[desired_order]
final_combined_acc.head()

,subzone_name,subzone_code,geog_boundary,planning_area_name,area,priv_to_public_ratio,total_bus_tap_in_volume,total_bus_tap_out_volume,total_buses_per_hour,num_bus_stops,...,fare_from_east region_to_mrt,duration_from_north-east region_to_mrt,walking_dist_from_north-east region_to_mrt,fare_from_north-east region_to_mrt,duration_from_north region_to_mrt,walking_dist_from_north region_to_mrt,fare_from_north region_to_mrt,duration_from_west region_to_mrt,walking_dist_from_west region_to_mrt,fare_from_west region_to_mrt
0,ADMIRALTY,SBSZ05,"POLYGON Z ((103.82855 1.45878 0, 103.82871 1.4...",SEMBAWANG,1.270958,0.623183,137532.182430,171045.739431,209.782652,16.522966,...,2.464000,4866.400000,1791.491000,2.360000,6217.2,1662.159000,2.47,5385.108333,2116.052,2.38973
1,AIRPORT ROAD,PLSZ04,"POLYGON Z ((103.90133 1.35659 0, 103.90136 1.3...",PAYA LEBAR,0.486377,0.438736,46088.367695,46152.104231,63.222531,6.168052,...,2.464000,4866.400000,1791.491000,2.360000,6217.2,1662.159000,2.47,5385.108333,2116.052,2.38973
2,ALEXANDRA HILL,BMSZ07,"POLYGON Z ((103.81445 1.28547 0, 103.81433 1.2...",BUKIT MERAH,1.037840,0.313048,408694.021923,499763.616300,472.013995,23.124952,...,2.464000,4866.400000,1791.491000,2.360000,6217.2,1662.159000,2.47,5385.108333,2116.052,2.38973
3,ALEXANDRA NORTH,BMSZ06,"POLYGON Z ((103.8174 1.29433 0, 103.81787 1.29...",BUKIT MERAH,0.295823,0.258394,166908.374821,135139.375783,274.234949,13.521600,...,2.464000,4866.400000,1791.491000,2.360000,6217.2,1662.159000,2.47,5385.108333,2116.052,2.38973
4,ALJUNIED,GLSZ04,"POLYGON Z ((103.89127 1.32132 0, 103.89153 1.3...",GEYLANG,2.982971,0.518057,518701.385040,534555.599716,445.487025,15.085630,...,1.683333,2465.333333,1133.783333,1.836667,4263.0,1240.456667,2.25,3973.666667,1229.460,2.15000


In [655]:
# Run to download final_combined_acc.csv
#final_combined_acc.to_csv("../data/final_combined_acc_v2.csv")

# **Hourly Data**
- Specially to track the fluctuations in public transport accessibility across hours in a day

In [656]:
# Assume subzone DataFrame is already defined
subzone_names = subzone["subzone_name"].unique()
hours = range(24)
year_months = ["2024-11", "2024-12", "2025-01"]

# Generate MultiIndex with subzone, hour, and year_month
pt_hourly = pd.MultiIndex.from_product([subzone_names, hours, year_months], 
                                       names=["subzone_name", "hour", "YEAR_MONTH"])

# Convert to DataFrame
pt_hourly = pd.DataFrame(index=pt_hourly).reset_index()

# Display result
pt_hourly.head()


,subzone_name,hour,YEAR_MONTH
0,DEPOT ROAD,0,2024-11
1,DEPOT ROAD,0,2024-12
2,DEPOT ROAD,0,2025-01
3,DEPOT ROAD,1,2024-11
4,DEPOT ROAD,1,2024-12


In [657]:
bus_pvod_volume_new

,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,ORIGIN_PT_CODE,DESTINATION_PT_CODE,TOTAL_TRIPS,BusStopCode_x,destination_subzone,BusStopCode_y,origin_subzone
0,2024-11,WEEKENDS/HOLIDAY,8,BUS,66331,70259,11,70259.0,KAMPONG UBI,66331.0,SERANGOON NORTH
1,2024-11,WEEKDAY,14,BUS,20289,17319,7,17319.0,CLEMENTI WEST,20289.0,WEST COAST
2,2024-11,WEEKDAY,14,BUS,82049,60219,3,60219.0,KALLANG WAY,82049.0,GEYLANG EAST
3,2024-11,WEEKDAY,11,BUS,55259,60081,3,60081.0,POTONG PASIR,55259.0,YIO CHU KANG NORTH
4,2024-11,WEEKDAY,18,BUS,84659,94079,1,94079.0,BAYSHORE,84659.0,BEDOK NORTH
...,...,...,...,...,...,...,...,...,...,...,...
17058078,2025-01,WEEKENDS/HOLIDAY,14,BUS,77201,75379,5,75379.0,TAMPINES WEST,77201.0,PASIR RIS WEST
17058079,2025-01,WEEKDAY,8,BUS,49069,49019,1,49019.0,PANG SUA,49069.0,RESERVOIR VIEW
17058080,2025-01,WEEKDAY,23,BUS,8019,6151,1,6151.0,ROBERTSON QUAY,8019.0,FORT CANNING
17058081,2025-01,WEEKENDS/HOLIDAY,13,BUS,54261,54581,192,54581.0,CHENG SAN,54261.0,CHENG SAN


In [658]:
bus_hourly_inbound = bus_pvod_volume_new.groupby(["YEAR_MONTH","TIME_PER_HOUR", "destination_subzone"], as_index=False)["TOTAL_TRIPS"].mean().rename(columns = {"destination_subzone":"subzone_name", 
                                                                                                                                                  "TOTAL_TRIPS":"total_bus_trips_inbound",
                                                                                                                                                  "TIME_PER_HOUR":"hour"
                                                                                                                                                  })
bus_hourly_outbound = bus_pvod_volume_new.groupby(["YEAR_MONTH","TIME_PER_HOUR", "origin_subzone"], as_index=False)["TOTAL_TRIPS"].mean().rename(columns = {"origin_subzone":"subzone_name", 
                                                                                                                                              "TOTAL_TRIPS":"total_bus_trips_outbound",
                                                                                                                                              "TIME_PER_HOUR":"hour"
                                                                                                                                              })

In [659]:
bus_hourly_inbound

,YEAR_MONTH,hour,subzone_name,total_bus_trips_inbound
0,2024-11,0,ADMIRALTY,4.006579
1,2024-11,0,ALEXANDRA HILL,3.092867
2,2024-11,0,ALEXANDRA NORTH,1.562500
3,2024-11,0,ALJUNIED,4.294350
4,2024-11,0,ANAK BUKIT,3.323113
...,...,...,...,...
19794,2025-01,23,YISHUN SOUTH,19.479943
19795,2025-01,23,YISHUN WEST,13.251034
19796,2025-01,23,YUHUA EAST,7.344930
19797,2025-01,23,YUHUA WEST,6.881393


In [660]:
train_hourly_outbound = train_pvod_volume_new.groupby(["YEAR_MONTH","TIME_PER_HOUR", "origin_subzone"], as_index=False)["TOTAL_TRIPS"].mean().rename(columns = {"origin_subzone": "subzone_name", 
                                                                                                                                              "TOTAL_TRIPS": "total_train_trips_outbound",
                                                                                                                                              "TIME_PER_HOUR": "hour"
                                                                                                                                              })
train_hourly_inbound = train_pvod_volume_new.groupby(["YEAR_MONTH","TIME_PER_HOUR", "destination_subzone"], as_index=False)["TOTAL_TRIPS"].mean().rename(columns = {"destination_subzone": "subzone_name", 
                                                                                                                                              "TOTAL_TRIPS": "total_train_trips_inbound",
                                                                                                                                              "TIME_PER_HOUR": "hour"
                                                                                                                                              })

In [661]:
pt_hourly = pt_hourly.merge(bus_hourly_inbound, on = ("subzone_name", "hour", "YEAR_MONTH"), how = "left")
pt_hourly = pt_hourly.merge(bus_hourly_outbound, on = ("subzone_name", "hour","YEAR_MONTH"), how = "left")

In [662]:
pt_hourly

,subzone_name,hour,YEAR_MONTH,total_bus_trips_inbound,total_bus_trips_outbound
0,DEPOT ROAD,0,2024-11,2.933333,2.205128
1,DEPOT ROAD,0,2024-12,3.204301,2.045455
2,DEPOT ROAD,0,2025-01,2.382353,1.857143
3,DEPOT ROAD,1,2024-11,NaN,NaN
4,DEPOT ROAD,1,2024-12,NaN,NaN
...,...,...,...,...,...
23899,TUAS VIEW EXTENSION,22,2024-12,22.757576,12.160714
23900,TUAS VIEW EXTENSION,22,2025-01,20.880000,10.846154
23901,TUAS VIEW EXTENSION,23,2024-11,15.714286,3.085714
23902,TUAS VIEW EXTENSION,23,2024-12,14.489796,4.281250


In [663]:
pt_hourly = pt_hourly.merge(train_hourly_inbound, on = ("subzone_name", "hour", "YEAR_MONTH"), how = "left")
pt_hourly = pt_hourly.merge(train_hourly_outbound, on = ("subzone_name", "hour", "YEAR_MONTH"), how = "left")

In [664]:
pt_hourly.fillna(0, inplace = True)
pt_hourly.head(23)

,subzone_name,hour,YEAR_MONTH,total_bus_trips_inbound,total_bus_trips_outbound,total_train_trips_inbound,total_train_trips_outbound
0,DEPOT ROAD,0,2024-11,2.933333,2.205128,0.0,0.0
1,DEPOT ROAD,0,2024-12,3.204301,2.045455,0.0,0.0
2,DEPOT ROAD,0,2025-01,2.382353,1.857143,0.0,0.0
3,DEPOT ROAD,1,2024-11,0.000000,0.000000,0.0,0.0
4,DEPOT ROAD,1,2024-12,0.000000,0.000000,0.0,0.0
5,DEPOT ROAD,1,2025-01,0.000000,0.000000,0.0,0.0
6,DEPOT ROAD,2,2024-11,0.000000,0.000000,0.0,0.0
7,DEPOT ROAD,2,2024-12,0.000000,0.000000,0.0,0.0
8,DEPOT ROAD,2,2025-01,0.000000,0.000000,0.0,0.0
9,DEPOT ROAD,3,2024-11,0.000000,0.000000,0.0,0.0


In [665]:
#pt_hourly.to_csv("../data/pt_hourly.csv")

In [666]:
#subzone.to_csv("../data/subzone_final.csv")